# Sign In/ Sign Up

In [1]:
import asyncio
from playwright.async_api import async_playwright

nvidia_url = "https://nvidia.wd5.myworkdayjobs.com/en-US/NVIDIAExternalCareerSite/job/US%2C-CA%2C-Santa-Clara/Senior-AI-and-ML-Engineer---AI-for-Networking_JR2000376/apply/applyManually?q=ml+enginer"
salesforce_url = "https://salesforce.wd12.myworkdayjobs.com/en-US/External_Career_Site/job/Singapore---Singapore/Senior-Manager--Solution-Engineering--Philippines-_JR301876/apply/applyManually"
hitachi_url = "https://hitachi.wd1.myworkdayjobs.com/en-US/hitachi/job/Krakow%2C-Lesser-Poland%2C-Poland/Internship---NET-Developer_R0071999/apply/applyManually"
icf_url = "https://icf.wd5.myworkdayjobs.com/en-US/ICFExternal_Career_Site/job/Reston%2C-VA/Senior-Paid-Search-Manager_R2502057/apply/applyManually"
harris_url = "https://harriscomputer.wd3.myworkdayjobs.com/en-US/1/job/Florida%2C-United-States/Vice-President-of-Sales_R0030918/apply/applyManually"

url = harris_url

playwright_instance = await async_playwright().start()
browser = await playwright_instance.chromium.launch(
    headless=False,
    slow_mo=100,
)

context = await browser.new_context(
    viewport={'width': 1280, 'height': 720}
)

page = await context.new_page()
await page.goto(url, wait_until='networkidle', timeout=30000)
print("Browser opened and navigated to the URL.")

Browser opened and navigated to the URL.


In [2]:
# take user input of sign in or sign up
user_choice = int(input("Enter 1 for sign in or 2 for sign up: "))

print(user_choice)

1


### Sign Up

In [3]:
import json

user_path = 'data/user_profile.json'

with open(user_path, 'r') as f:
    user_data = json.load(f)

email = user_data['personal_information']['email']
password = user_data['personal_information']['password']

if user_choice == 2:
    # Fill email
    email_input = await page.query_selector('input[data-automation-id="email"]')

    if email_input:
        await email_input.fill(email)
        print(f" Filled email: {email}")
    else:
        print(" Email input not found.")

    # Fill password
    password_input = await page.query_selector('input[data-automation-id="password"]')
    if password_input:
        await password_input.fill(password)
        print(" Filled password.")
    else:
        print(" Password input not found.")

    # Fill verify password
    verify_password_input = await page.query_selector('input[data-automation-id="verifyPassword"]')
    if verify_password_input:
        await verify_password_input.fill(password)
        print(" Filled verify password.")
    else:
        print(" Verify password input not found.")

    # Check the create account checkbox
    checkbox = await page.query_selector('input[data-automation-id="createAccountCheckbox"]')
    if checkbox:
        checked = await checkbox.is_checked()
        if not checked:
            await checkbox.check()
        print(" Checked create account checkbox.")
    else:
        print(" Create account checkbox not found.")

    # Click the submit button
    await asyncio.sleep(1)  # Wait for any potential UI updates
    submit_btn = await page.query_selector('div[aria-label="Create Account"]')

    if submit_btn:
        await submit_btn.click()
        print(" Clicked create account submit button.")

### Sign In

In [4]:
import json

user_path = 'data/user_profile.json'

with open(user_path, 'r') as f:
    user_data = json.load(f)

if user_choice == 1:

    sign_in_link_selector = 'button[data-automation-id="signInLink"]'

    sign_in_link = await page.query_selector(sign_in_link_selector)
    if sign_in_link:
        await sign_in_link.click()
        print("Clicked sign in link.")
    else:
        print(" Sign in link not found.")

    email = user_data['personal_information']['email']
    password = user_data['personal_information']['password']

    # Fill email
    email_input = await page.query_selector('input[data-automation-id="email"]')

    if email_input:
        await email_input.fill(email)
        print(f" Filled email: {email}")
    else:
        print(" Email input not found.")

    # Fill password
    password_input = await page.query_selector('input[data-automation-id="password"]')
    if password_input:
        await password_input.fill(password)
        print(" Filled password.")
    else:
        print(" Password input not found.")

    await asyncio.sleep(1)  # Wait for any potential UI updates
    submit_btn = await page.query_selector('div[aria-label="Sign In"]')
    if submit_btn:
        await submit_btn.click()
        print("Clicked sign in submit button.")

Clicked sign in link.
 Filled email: lmei123111115191@gmail.com
 Filled password.
Clicked sign in submit button.


In [5]:
async def load_user_profile():
    """Load user profile data with error handling"""
    user_path = 'data/user_profile.json'
    try:
        with open(user_path, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"User profile file not found: {user_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Invalid JSON in user profile file: {user_path}")
        return {}
    
import json
import openai
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

API_KEY = os.getenv("OPENAI_API_KEY")

client = openai.AsyncOpenAI(api_key=API_KEY)

await asyncio.sleep(12)  # Wait for any potential UI updates

# My Information

In [6]:
# My Information - Refactored with Modular Approach

async def get_nearest_label_text(element, page):
    """Get the nearest label text for a form element by checking multiple DOM patterns"""
    try:
        # Try explicit <label for="">
        element_id = await element.get_attribute('id')
        if element_id and element_id != "unknown":
            label_elem = await page.query_selector(f'label[for="{element_id}"]')
            if label_elem:
                label_text = await label_elem.text_content()
                if label_text:
                    return label_text.replace('*', '').strip()
        
        # Try parent label (element wrapped in label)
        parent_label_handle = await element.evaluate_handle('el => el.closest("label")')
        parent_label = parent_label_handle.as_element() if parent_label_handle else None
        if parent_label:
            label_text = await parent_label.text_content()
            if label_text:
                return label_text.replace('*', '').strip()
        
        # Try nearest formField label in parent containers
        form_field_label = await element.evaluate('''
            el => {
                let cur = el.parentElement;
                let depth = 0;
                while (cur && depth < 10) {
                    if (cur.tagName.toLowerCase() === "div" &&
                        cur.getAttribute("data-automation-id")?.startsWith("formField-")) {
                        const lbl = cur.querySelector("label span, label");
                        if (lbl && lbl.textContent) return lbl.textContent.trim();
                    }
                    cur = cur.parentElement;
                    depth++;
                }
                return null;
            }
        ''')
        if form_field_label:
            return form_field_label.replace('*', '').strip()
        
        # Try aria-labelledby attribute
        aria_labelledby = await element.get_attribute('aria-labelledby')
        if aria_labelledby:
            label_element = await page.query_selector(f'#{aria_labelledby}')
            if label_element:
                label_text = await label_element.text_content()
                if label_text:
                    return label_text.replace('*', '').strip()
        
        # Try aria-label attribute
        aria_label = await element.get_attribute('aria-label')
        if aria_label:
            return aria_label.replace('*', '').strip()
        
        # Try placeholder as fallback
        placeholder = await element.get_attribute('placeholder')
        if placeholder:
            return placeholder.replace('*', '').strip()
        
        return None
        
    except Exception as e:
        print(f"Error getting label for element: {e}")
        return None

async def get_group_label_and_aria(element, page):
    """
    Returns a tuple: (group_label, aria_labelledby)
    - group_label: The label text of the group containing the element (legend, label, or aria-labelledby)
    - aria_labelledby: The value of the aria-labelledby attribute of the group (if present)
    """
    result = await element.evaluate('''
        el => {
            let group = el.closest("fieldset, [role='group']");
            let aria_labelledby = null;
            let label_text = null;
            if (group) {
                // Try legend
                let legend = group.querySelector("legend");
                if (legend && legend.textContent) label_text = legend.textContent.trim();
                // Try aria-labelledby
                let labelledby = group.getAttribute("aria-labelledby");
                if (labelledby) {
                    aria_labelledby = labelledby;
                    let labelEl = document.getElementById(labelledby);
                    if (labelEl && labelEl.textContent) label_text = labelEl.textContent.trim();
                }
                // Try label inside group
                if (!label_text) {
                    let label = group.querySelector("label");
                    if (label && label.textContent) label_text = label.textContent.trim();
                }
            }
            // Try parent div with aria-labelledby if not found
            if (!label_text) {
                let cur = el.parentElement;
                let depth = 0;
                while (cur && depth < 15) {
                    let labelledby = cur.getAttribute && cur.getAttribute("aria-labelledby");
                    if (labelledby) {
                        aria_labelledby = labelledby;
                        let labelEl = document.getElementById(labelledby);
                        if (labelEl && labelEl.textContent) {
                            label_text = labelEl.textContent.trim();
                            break;
                        }
                    }
                    cur = cur.parentElement;
                    depth++;
                }
            }
            return {label_text, aria_labelledby};
        }
    ''')
    return result.get('label_text'), result.get('aria_labelledby')

async def get_ai_response_for_panel(current_data, panel_elements, client):
    """Get AI response for form fields using OpenAI"""
    form_fields = []
    key_mapping = {}

    for el in panel_elements:
        full_key = f"[{el['question']}, {el['input_id']}, {el['input_type']}, {el['aria_labelledby']}, {el['input_tag']}]"
        
        form_fields.append({
            "full_key": full_key,
            "question": el['question'],
            "input_id": el['input_id'],
            "input_type": el['input_type'],
            "input_tag": el['input_tag'],
            "aria_labelledby": el['aria_labelledby'],
            "options": el['options'], 
            "placeholder": el.get('placeholder'),
            "required": el.get('required'),
            "role": el.get('role')
        })
        
        key_mapping[full_key] = el

    prompt = f"""
You are helping fill a job application form.
You are mapping user profile data to a web form.

You are given:
- An Entry from user profile data (JSON)
- A list of form fields from the application panel (including labels, field types, and available options if there is dropdown for the element)

Return a JSON dictionary mapping the EXACT "full_key" values to appropriate values. Use the user data to fill the values. If a field is not relevant, map it to "SKIP".

CRITICAL: You MUST use the EXACT "full_key" value as the key in your response JSON. Do NOT use just the question text.

IMPORTANT RULES:
- For fields with "options" not None:
  - You MUST select ONLY from the list of provided OPTIONS (case-sensitive)
  - If the user data is longer (e.g., "Bachelor of Engineering in Computer Science") and options are shorter (e.g., "BS"), choose the CLOSEST MATCH based on meaning
  - If no match is appropriate, use "SKIP"
- For date fields: Month should be number format (e.g., "01" for January), year should be "YYYY" format
- For date-related fields (e.g. type="spinbutton" or input_id includes "Month" or "Year"):
  - Use "MM" format for months (e.g., "01" for January)
  - Use "YYYY" format for years (e.g., "2022")
  - Match "start date", "end date", "graduation date", etc., with the corresponding data from user profile
- For text fields: Keep responses concise and relevant
- Match options exactly as they appear in the options list (case-sensitive) when options is not None
- After filling the form, if a field for save and continue is present, respond with yes to save the form

SPECIAL HANDLING FOR SKILLS/MULTI-VALUE FIELDS:
- For fields related to skills, technologies, competencies, or any field that should contain multiple items:
  - Return an ARRAY of strings instead of a single comma-separated string
  - Each skill/technology should be a separate string in the array
  - Example: ["C#", "TypeScript", "Java", "SQL", "HTML5", "CSS3", "Python"] instead of "C#, TypeScript, Java, SQL, HTML5, CSS3, Python"
- Identify skills fields by keywords in the question like: "skills", "technologies", "competencies", "tools", "programming languages", etc.

Data from User Profile:
{json.dumps(current_data, indent=2)}

Form Fields:
{json.dumps(form_fields, indent=2)}

Example response format:
{{
  "[School or University*, unknown, text, Education-(Optional)-2-panel, input]": "University Name",
  "[Degree*, unknown, button, Education-(Optional)-2-panel, button]": "MS",
  "[Field of Study, unknown, unknown, Education-(Optional)-2-panel, input]": "Computer Science",
  "[Type to Add Skills, unknown, unknown, Skills-section, input]": ["C#", "TypeScript", "Java", "SQL", "HTML5", "CSS3", "Python", ".NET Core", "Angular 2+", "RxJS", "Entity Framework", "React", "Redux", "Bootstrap 4"]
}}

Respond ONLY with a valid JSON object using the exact "full_key" values as keys.
"""
    
    try:
        response = await client.chat.completions.create(
            model="o4-mini",
            messages=[{"role": "user", "content": prompt}]
        )
        content = response.choices[0].message.content.strip()
        ai_response = json.loads(content)
        return ai_response, key_mapping
    except Exception as e:
        print(f"Error in get_ai_response_for_panel: {e}")
        return {}, {}

async def get_listbox_options(input_el, page, full_page):
    """Extract options from a listbox by clicking it and reading the <li> elements"""
    await input_el.click()
    await asyncio.sleep(1)
    
    selector = 'div[visibility="opened"]'
    options = []
    listbox_container = await full_page.query_selector(selector)
    
    if listbox_container:
        li_elements = await listbox_container.query_selector_all('li[role="option"]')
        for li in li_elements:
            text = await li.text_content()
            if text and text.strip():
                options.append(text.strip())
            else:
                div_text = await li.query_selector('div')
                if div_text:
                    nested_text = await div_text.text_content()
                    if nested_text and nested_text.strip():
                        options.append(nested_text.strip())

    await input_el.click()
    await asyncio.sleep(0.5)
    return options

async def fill_listbox_element(input_el, page, response, full_page):
    """Fill a listbox/combobox element with the given response"""
    await input_el.click()
    await asyncio.sleep(0.5)
    
    listbox_selectors = ['div[visibility="opened"]']
    
    for selector in listbox_selectors:
        listbox = await full_page.query_selector(selector)
        if listbox:
            li_elements = await listbox.query_selector_all('li')
            for li in li_elements:
                text = await li.text_content()
                if text.lower() == response.lower():
                    await li.click()
                    return True
                else:
                    div_element = await li.query_selector('div')
                    if div_element:
                        div_text = await div_element.text_content()
                        if div_text and response.lower() in div_text.lower():
                            await li.click()
                            await asyncio.sleep(0.2)
                            return True
            break
    
    return False

async def get_element_options(input_el, page, input_tag, input_type, full_page):
    """Get options for different input types"""
    options = None
    
    if input_tag == "button" or await input_el.get_attribute('role') == 'combobox':
        aria_haspopup = await input_el.get_attribute('aria-haspopup')
        if aria_haspopup == "listbox":
            options = await get_listbox_options(input_el, page, full_page)
    
    return options

async def fill_element(full_page, input_el, input_id, input_type, input_tag, response, options=None):

    try:
        if response == "SKIP":
            print(f"Skipping input {input_id} as per AI {response}")
            return

        if input_tag == "input" and input_type == "file":
            await input_el.set_input_files([response])
            return

        # Check if element is inside a multiSelectContainer
        is_multi_select = await input_el.evaluate('''
            el => {
                let cur = el.parentElement;
                let depth = 0;
                while (cur && depth < 10) {
                    if (cur.getAttribute("data-automation-id")?.includes("multiSelectContainer")) {
                        return true;
                    }
                    cur = cur.parentElement;
                    depth++;
                }
                return false;
            }
        ''')
        print(f"is_multi_select: {is_multi_select}")

        # TEXT inputs or TEXTAREAs
        if input_tag in ["input", "textarea"] and not await input_el.get_attribute("dir") and input_type not in ["radio", "checkbox", "spinbutton"]:
            await input_el.fill(response)
            return

        # MultiInputContainer elements - Handle differently based on container type
        elif is_multi_select:
            print(f"Entering the condition for MultiInputContainer for {input_id}")
            if isinstance(response, list):
                pass
            else:
                response = [response]      
            print(f"Filling MultiInputContainer for {input_id} with responses: {response}")

            container_handle = await input_el.evaluate_handle('''
                        el => {
                            let cur = el.parentElement;
                            let depth = 0;
                            while (cur && depth < 10) {
                                if (cur.getAttribute("data-automation-id")?.includes("multiSelectContainer")) {
                                    return cur;  // Return the container div itself
                                }
                                cur = cur.parentElement;
                                depth++;
                            }
                            return null;
                        }
                    ''')
     

            input_clickable = container_handle.as_element() if container_handle else None 
            print(f"Clicked the div with multiSelectContainer for {input_id}")
            for res in response:
                await input_clickable.click()
                await asyncio.sleep(0.5)
                await input_clickable.fill(res)
                await input_clickable.press('Enter')
                await asyncio.sleep(1)

                prompt_options = await full_page.query_selector_all(
                    'div[data-automation-id="promptOption"], div[data-automation-id="promptLeafNode"]'
                )
                # the prompt_options tagName shouldn't be <p (as in paragraph) but <span> or <div>
                print("Before : ", prompt_options)
                # prompt_options = [opt for opt in prompt_options if not await opt.evaluate('(el) => el.tagName.toLowerCase()') == 'p']
                # print("After : ", prompt_options)
                if prompt_options:
                    first_option = prompt_options[0]
                    await first_option.click()
                    print(f"Selected first promptOption: {await first_option.text_content()}")
                


        elif input_type == "checkbox":
            is_checked = await input_el.is_checked()
            should_be_checked = str(response).lower() in ["yes", "true", "1", "check", "selected"] or response is True
            
            if should_be_checked and not is_checked:
                await input_el.click()
            elif not should_be_checked and is_checked:
                await input_el.click()
            return

        # SELECT dropdowns
        elif input_tag == "select":
            if options and response not in options:
                print(f"[WARN] Response '{response}' not in select options: {options}")
                return
            await input_el.select_option(label=response)

        # LISTBOX/COMBOBOX buttons
        elif input_tag == "button":
            role = await input_el.get_attribute('role')
            aria_haspopup = await input_el.get_attribute('aria-haspopup')
            if aria_haspopup == "listbox":
                if options and not any(response.lower() in opt.lower() for opt in options):
                    print(f"[WARN] Response '{response}' not found in listbox options: {options}")
                    return
                
                success = await fill_listbox_element(input_el, page, response, full_page)
                if not success:
                    print(f"[WARN] Failed to select '{response}' from listbox")

        # DATE (spinbutton)
        elif input_type == "spinbutton":
            await input_el.fill(response)

        # CHECKBOX or RADIO
        elif input_type in ["checkbox", "radio"]:
            if response.lower() in ["yes", "true", "1", "check", "selected"]:
                is_checked = await input_el.is_checked()
                if not is_checked:
                    await input_el.click()

        else:
            print(f"[WARN] Unhandled input type/tag: {input_type} / {input_tag}")

    except Exception as e:
        print(f"[ERROR] fill_element failed for {input_tag} / {input_type}: {e}")
        
# Main My Information Processing Function
async def process_my_information_section(page):
    """Process the My Information section using modular approach with while loop"""
    
    await asyncio.sleep(5)  # Wait for page to load
    
    main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')
    INPUT_SELECTOR = 'button, input, select, textarea, [role="button"]'
    
    i = 0
    prev_answered_question = None
    
    while True:
        inputs = await main_page.query_selector_all(INPUT_SELECTOR)
        
        if i >= len(inputs):
            break
            
        input_el = inputs[i]
        
        # Get element information
        input_id = await input_el.get_attribute('data-automation-id') or await input_el.get_attribute('aria-haspopup') or 'unknown'
        group_label, aria_labelledby = await get_group_label_and_aria(input_el, page)
        question = group_label or 'UNLABELED'
        
        # Get more specific question label
        question = await get_nearest_label_text(input_el, page) or question
        
        # Skip duplicate questions
        if question != 'UNLABELED' and question == prev_answered_question:
            print(f"⏩ Skipping duplicate question at index {i}: '{question}'")
            i += 1
            continue
        
        # Skip navigation buttons
        if input_id == "pageFooterBackButton" or input_id == "backToJobPosting":
            i += 1
            continue
        
        print("--------------------------------")
        print(f"Input ID: {input_id}, Question: {question}, aria-labelledby: {aria_labelledby or 'None'}")
        
        # Handle Next button
        if input_id == "pageFooterNextButton":
            print("Clicking Next button")
            await input_el.click()
            break
        
        # Process form elements
        else:
            user_data = await load_user_profile()
            section = await main_page.query_selector(f'div[aria-labelledby="{aria_labelledby}"]') if aria_labelledby else main_page
            
            # Get element properties
            input_type = await input_el.get_attribute('type') or 'unknown'
            role = await input_el.get_attribute('role')
            placeholder = await input_el.get_attribute('placeholder')
            required = await input_el.get_attribute('required')
            
            if role == "spinbutton":
                input_type = "spinbutton"
            
            tag_name = await input_el.evaluate("(el) => el.tagName.toLowerCase()")
            if tag_name and tag_name.lower() == 'textarea':
                input_type = 'textarea'
            input_tag = tag_name
            
            # Get options for relevant input types
            options = await get_element_options(input_el, section, input_tag, input_type, page)
            
            print(f"Processing form element: {question}")
            
            # Create element info for AI processing
            element_info = {
                'question': question or 'UNLABELED',
                'aria_labelledby': aria_labelledby,
                'input_type': input_type,
                'input_tag': input_tag,
                'input_id': input_id,
                'options': options,
                'placeholder': placeholder,
                'required': required,
                'role': role
            }
            
            print(f"Element info: {element_info}")

            if(await input_el.get_attribute('dir') != 'ltr'):
                print(f"Skipping element {input_id} with dir={await input_el.get_attribute('dir')}")
                i += 1
                continue
            
            # Get AI response
            full_key = f"[{element_info['question']}, {element_info['input_id']}, {element_info['input_type']}, {element_info['aria_labelledby']}, {element_info['input_tag']}]"
            ai_values, _ = await get_ai_response_for_panel(user_data, [element_info], client)
            response = ai_values.get(full_key, 'SKIP')
            
            print(f"AI response for field '{question}': {response}")
            
            # Fill the element
            await fill_element(
                page,
                input_el, 
                input_id, 
                input_type, 
                input_tag, 
                response,
                options
            )
            
            if question != 'UNLABELED':
                prev_answered_question = question
        
        i += 1
    
    print("My Information section processing completed.")

# Execute the My Information section
await process_my_information_section(page)

--------------------------------
Input ID: listbox, Question: How Did You Hear About Us?, aria-labelledby: source-section
Processing form element: How Did You Hear About Us?
Element info: {'question': 'How Did You Hear About Us?', 'aria_labelledby': 'source-section', 'input_type': 'button', 'input_tag': 'button', 'input_id': 'listbox', 'options': ['Select One', '*I know someone at the company', 'Altera', "America's Job Exchange", 'BC Jobs', 'Blast Network', 'Career Builder', 'Compliance_HireAHero', 'Compliance_JOFDAV', 'Compliance_recruitABILITY', 'Compliance_USDiversity', 'CSI Transfer', 'Glassdoor', 'Google', 'Harris Career Website', 'Indeed', 'JobBoom', 'Jobillico', 'LinkedIn', 'Monster', 'SEEK.com.au', 'StackOverflow', 'States_Compliance'], 'placeholder': None, 'required': None, 'role': None}
Skipping element listbox with dir=None
--------------------------------
Input ID: unknown, Question: How Did You Hear About Us?, aria-labelledby: source-section
Processing form element: How Di

TargetClosedError: ElementHandle.text_content: Target page, context or browser has been closed
Call log:
  - waiting for locator(":scope")


### How did you hear about us?

#### TODO : Extract all options and save them

In [ ]:
how_did_you_hear_selector = page.locator('div[data-automation-id="formField-source"] div[data-automation-id="multiselectInputContainer"]')

if await how_did_you_hear_selector.count() > 0:
    await how_did_you_hear_selector.click()
    await asyncio.sleep(1)  # Wait for any potential UI updates
    import random
    # Keep clicking until no new promptLeafNode options appear
    max_attempts = 3
    attempt = 0

    while attempt < max_attempts:
        prompt_options = await page.query_selector_all('div[data-automation-id="promptLeafNode"]')
        
        for opt in prompt_options:
            print(await opt.text_content())
        
        if prompt_options:
            selected_option = random.choice(prompt_options)
            
            for opt in prompt_options:
                print(await opt.text_content())

            await selected_option.click()
            print("Clicked on a random prompt option.")
            await asyncio.sleep(2)  # Wait for any potential UI updates
            # Check if new options appeared after clicking
            new_options = await page.query_selector_all('div[data-automation-id="promptLeafNode"]')

            # If no new options or same as before, break
            if len(new_options) == 0 or len(new_options) == len(prompt_options):
                break
        else:
            print(" No prompt options found.")
        attempt += 1

    await asyncio.sleep(1)  # Wait for any potential UI updates

### Radio

In [ ]:
no_radio = await page.query_selector('input[name="candidateIsPreviousWorker"][type="radio"][value="false"]')
if no_radio:
    checked = await no_radio.is_checked()
    if not checked:
        await no_radio.check()
        print(" Selected 'No' for previous worker.")
    else:
        print("'No' already selected for previous worker.")
else:
    print(" Could not find 'No' radio button for previous worker.")

'No' already selected for previous worker.


### Country ListBox

In [ ]:
# # --- Country selection ---
# await asyncio.sleep(1)  # Wait for any potential UI updates
# country_button = await page.query_selector('button[aria-haspopup="listbox"][id*="country"]')
# if country_button:
#     await country_button.click()
#     print(" Opened country dropdown.")
#     await page.wait_for_selector('div[visibility="opened"] li[role="option"]', timeout=10000)
#     options = await page.query_selector_all('div[visibility="opened"] li[role="option"]')
#     # print("Country options:")
#     for i, option in enumerate(options, 1):
#         text = await option.text_content()
#         # print(f"{i}: {text.strip() if text else ''}")
#     # Select "United States of America" if present, else pick the first
#     selected = False
#     for option in options:
#         text = await option.text_content()
#         if text and "united states of america" in text.lower():
#             await option.click()
#             # print(f" Selected: {text.strip()}")
#             selected = True
#             break
#     if not selected and options:
#         await options[0].click()
#         print(f" Selected default: {await options[0].text_content()}")
# else:
#     print(" Country dropdown button not found.")

# # wait
# await asyncio.sleep(2)

 Opened country dropdown.


### ListBoxes

In [ ]:
import json
import openai
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

API_KEY = os.getenv("OPENAI_API_KEY")

client = openai.AsyncOpenAI(api_key=API_KEY)

async def get_best_option(user_data, question, options, client):
    prompt = f"""
You are mapping user profile data to a web form.
Given the following user data (JSON), a question, and a list of options, select the best option to answer the question. 
If none are suitable, respond with 'SKIP'.

Important Instruction:
- For a question based on how did you hear about us or similar, respond with the most appropriate answer based on the user's profile. DO NOT use "SKIP" for these questions.

User Data:
{json.dumps(user_data, indent=2)}

Question: {question or 'UNLABELED'}
Options: {options}

Respond ONLY with the exact option text from the list above, or 'SKIP'.
"""
    try:
        response = await client.chat.completions.create(
            model="gpt-4.1",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=30,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error getting best option: {e}")
        return "SKIP"

async def get_nearest_label_text(element, page):
    # Try explicit <label for="">
    element_id = await element.get_attribute('id')
    if element_id:
        label_elem = await page.query_selector(f'label[for="{element_id}"]')
        if label_elem:
            label_text = await label_elem.text_content()
            if label_text:
                return label_text.strip()
    # Try parent label
    parent_label_handle = await element.evaluate_handle('el => el.closest("label")')
    parent_label = parent_label_handle.as_element() if parent_label_handle else None
    if parent_label:
        label_text = await parent_label.text_content()
        if label_text:
            return label_text.strip()
    # Try nearest formField label
    form_field_label = await element.evaluate('''
        el => {
            let cur = el.parentElement;
            let depth = 0;
            while (cur && depth < 6) {
                if (cur.tagName.toLowerCase() === "div" &&
                    cur.getAttribute("data-automation-id")?.startsWith("formField-")) {
                    const lbl = cur.querySelector("label span, label");
                    if (lbl && lbl.textContent) return lbl.textContent.trim();
                }
                cur = cur.parentElement;
                depth++;
            }
            return null;
        }
    ''')
    if form_field_label:
        return form_field_label.strip()
    # Fallback: previous or parent label
    return None

listboxes = await page.query_selector_all('div[data-automation-id="applyFlowPage"] button[aria-haspopup="listbox"]')
for i, listbox in enumerate(listboxes, 1):
    listbox_id = await listbox.get_attribute('id') or ""
    listbox_name = await listbox.get_attribute('name') or ""
    print(listbox_id, listbox_name)
    # if listbox_name == "country":
    #     continue
    question = await get_nearest_label_text(listbox, page)
    try:
        await listbox.click()
        print(f" Opened listbox {i} | Question: {question or 'UNLABELED'}")
        await page.wait_for_selector('div[visibility="opened"] li[role="option"]', timeout=5000)
        options_els = await page.query_selector_all('div[visibility="opened"] li[role="option"]')
        options = [((await el.text_content()) or "").strip() for el in options_els]
        print(f"Options: {options}")

        if options:
            best_option = await get_best_option(user_data, question, options, client)
            print(f"AI selected: {best_option}")
            if best_option and best_option != "SKIP":
                # Find and click the matching option
                for el, text in zip(options_els, options):
                    if text.lower() == best_option.lower():
                        await el.click()
                        print(f" Selected: {text} for listbox {i} | Question: {question or 'UNLABELED'}")
                        break
                else:
                    print(f" No exact match found for AI-selected option: {best_option}")
            else:
                print(f"AI chose to skip this listbox.")
        else:
            print(f" No options found for listbox {i} | Question: {question or 'UNLABELED'}")
        await asyncio.sleep(1)
    except Exception as e:
        print(f"  Error processing listbox {i} | Question: {question or 'UNLABELED'}: {e}")
        await asyncio.sleep(1)

source--source source
 Opened listbox 1 | Question: How Did You Hear About Us?*
Options: ['Select One', '*I know someone at the company', 'Altera', "America's Job Exchange", 'BC Jobs', 'Blast Network', 'Career Builder', 'Compliance_HireAHero', 'Compliance_JOFDAV', 'Compliance_recruitABILITY', 'Compliance_USDiversity', 'CSI Transfer', 'Glassdoor', 'Google', 'Harris Career Website', 'Indeed', 'JobBoom', 'Jobillico', 'LinkedIn', 'Monster', 'SEEK.com.au', 'StackOverflow', 'States_Compliance']
AI selected: LinkedIn
 Selected: LinkedIn for listbox 1 | Question: How Did You Hear About Us?*
country--country country
 Opened listbox 2 | Question: Country*
Options: ['Select One', 'Afghanistan', 'Åland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'B

### Text inputs

In [ ]:
import json
import os
import asyncio
import openai
from playwright.async_api import async_playwright

client = openai.AsyncOpenAI(api_key=API_KEY)

async def load_user_profile():
    """Load user profile data with error handling"""
    user_path = 'data/user_profile.json'
    try:
        with open(user_path, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"User profile file not found: {user_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Invalid JSON in user profile file: {user_path}")
        return {}

async def get_nearest_question_label(element):
    """Helper to get nearest label/question for an input element"""
    try:
        # Check for label with 'for' attribute matching element id
        element_id = await element.get_attribute("id")
        if element_id:
            label_text = await element.evaluate(f'''
                el => {{
                    const lbl = document.querySelector('label[for="{element_id}"]');
                    return lbl ? lbl.textContent.trim() : null;
                }}
            ''')
            if label_text:
                return label_text.replace('*', '').strip()
        
        # Check for wrapping label
        wrapped_label = await element.evaluate('''
            el => {
                const lbl = el.closest("label");
                return lbl ? lbl.textContent.trim() : null;
            }
        ''')
        if wrapped_label:
            return wrapped_label.replace('*', '').strip()
        
        # Check for form field labels in parent containers
        form_field_label = await element.evaluate('''
            el => {
                let cur = el.parentElement;
                let depth = 0;
                while (cur && depth < 6) {
                    if (cur.tagName.toLowerCase() === "div" &&
                        cur.getAttribute("data-automation-id")?.startsWith("formField-")) {
                        const lbl = cur.querySelector("label span, label");
                        if (lbl && lbl.textContent) return lbl.textContent.trim();
                    }
                    cur = cur.parentElement;
                    depth++;
                }
                return null;
            }
        ''')
        if form_field_label:
            return form_field_label.replace('*', '').strip()
        
        return None
    except Exception as e:
        print(f"Error getting label for element: {e}")
        return None

async def get_ai_response(user_data, input_name, question):
    """Get AI response for form field value using OpenAI"""
    prompt = f"""
You are mapping user profile data to a web form.
Given the following user data (JSON) and the input field's question, return the best value for this input. For country, state names : return the full name (e.g., "United States of America" instead of "USA"). 

User Data:
{json.dumps(user_data, indent=2)}

Input Field:
- name/id: {input_name}
- question: {question or 'UNLABELED'}

Respond ONLY with the value to fill, or 'SKIP' if no suitable value or irrelevant question.
"""
    
    try:
        response = await client.chat.completions.create(
            # 4.1 nano model
            model="gpt-4.1-nano",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=50,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error getting AI response: {e}")
        return "SKIP"

async def fill_form(page):
    """Main function to fill form fields"""
    # Load user profile data
    user_data = await load_user_profile()
    if not user_data:
        print("No user data loaded, exiting...")
        return
    
    # Find all text input elements
    text_inputs = await page.query_selector_all(
        'input[type="text"], input[type="email"], input[type="password"], input[type="tel"], textarea'
    )
    
    if not text_inputs:
        print("No text input fields found on the page")
        return
    
    # ids and names for debugging
    for i, input_el in enumerate(text_inputs, 1):
        input_id = await input_el.get_attribute('id') or f'input_{i}'
        input_name = await input_el.get_attribute('name') or f'input_{i}'
        print(f"Input {i}: ID='{input_id}', Name='{input_name}'")
    
    print(f"Found {len(text_inputs)} input fields to process")
    
    for i, input_el in enumerate(text_inputs, 1):
        try:
            # Get field information
            question = await get_nearest_question_label(input_el)
            input_name = (await input_el.get_attribute('name') or 
                         await input_el.get_attribute('id') or 
                         f'input_{i}')
            
            print(f"Processing input {i}: {input_name} | Question: {question or 'UNLABELED'}")
            
            # Get AI-generated value
            value = await get_ai_response(user_data, input_name, question)
            print(f"AI suggested value for '{input_name}': {value}")
            # Fill the field if we have a valid value
            if value and value.upper() != "SKIP":
                # Check if element is still attached and visible
                is_visible = await input_el.is_visible()
                if is_visible:
                    await input_el.fill(value)
                    print(f"Filled '{input_name}' with '{value}'")
                else:
                    print(f"Skipped '{input_name}' (element not visible)")
            else:
                print(f"Skipped '{input_name}' (no suitable value found)")
                
        except Exception as e:
            print(f"Error processing input {i}: {e}")
            continue
        
        # Add small delay to avoid overwhelming the page
        await asyncio.sleep(0.1)

main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')
await fill_form(main_page)
await asyncio.sleep(1)  # Wait for any potential UI updates 

Input 1: ID='input_1', Name='input_1'
Input 2: ID='input_2', Name='input_2'
Input 3: ID='name--legalName--firstName', Name='legalName--firstName'
Input 4: ID='name--legalName--lastName', Name='legalName--lastName'
Input 5: ID='address--addressLine1', Name='addressLine1'
Input 6: ID='address--city', Name='city'
Input 7: ID='input_7', Name='input_7'
Input 8: ID='address--postalCode', Name='postalCode'
Input 9: ID='input_9', Name='input_9'
Input 10: ID='phoneNumber--phoneNumber', Name='phoneNumber'
Input 11: ID='phoneNumber--extension', Name='extension'
Found 11 input fields to process
Processing input 1: input_1 | Question: How Did You Hear About Us?
AI suggested value for 'input_1': SKIP
Skipped 'input_1' (no suitable value found)
Processing input 2: input_2 | Question: Country
AI suggested value for 'input_2': United States of America
Skipped 'input_2' (element not visible)
Processing input 3: legalName--firstName | Question: First Name
AI suggested value for 'legalName--firstName': Li

In [ ]:
save_and_continue_button = await page.locator('button[data-automation-id="pageFooterNextButton"]').click()

await asyncio.sleep(2)  # Wait for any potential UI updates

# Experience

In [ ]:
await asyncio.sleep(5)  # Wait for any potential UI updates

main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')

inputs = await main_page.query_selector_all('input, button, textarea, select')

async def get_group_label_and_aria(element, page):
    """
    Returns a tuple: (group_label, aria_labelledby)
    - group_label: The label text of the group containing the element (legend, label, or aria-labelledby)
    - aria_labelledby: The value of the aria-labelledby attribute of the group (if present)
    """
    result = await element.evaluate('''
        el => {
            let group = el.closest("fieldset, [role='group']");
            let aria_labelledby = null;
            let label_text = null;
            if (group) {
                // Try legend
                let legend = group.querySelector("legend");
                if (legend && legend.textContent) label_text = legend.textContent.trim();
                // Try aria-labelledby
                let labelledby = group.getAttribute("aria-labelledby");
                if (labelledby) {
                    aria_labelledby = labelledby;
                    let labelEl = document.getElementById(labelledby);
                    if (labelEl && labelEl.textContent) label_text = labelEl.textContent.trim();
                }
                // Try label inside group
                if (!label_text) {
                    let label = group.querySelector("label");
                    if (label && label.textContent) label_text = label.textContent.trim();
                }
            }
            // Try parent div with aria-labelledby if not found
            if (!label_text) {
                let cur = el.parentElement;
                let depth = 0;
                while (cur && depth < 15) {
                    let labelledby = cur.getAttribute && cur.getAttribute("aria-labelledby");
                    if (labelledby) {
                        aria_labelledby = labelledby;
                        let labelEl = document.getElementById(labelledby);
                        if (labelEl && labelEl.textContent) {
                            label_text = labelEl.textContent.trim();
                            break;
                        }
                    }
                    cur = cur.parentElement;
                    depth++;
                }
            }
            return {label_text, aria_labelledby};
        }
    ''')
    return result.get('label_text'), result.get('aria_labelledby')

for input_el in inputs:
    input_id = await input_el.get_attribute('data-automation-id') or 'unknown'
    group_label, aria_labelledby = await get_group_label_and_aria(input_el, page)
    question = group_label or 'UNLABELED'
    if input_id == "pageFooterBackButton" or input_id == "backToJobPosting":
        continue
    print(f"Input ID: {input_id}, Question: {question}, aria-labelledby: {aria_labelledby or 'None'}")

Input ID: unknown, Question: How Did You Hear About Us?*, aria-labelledby: source-section
Input ID: unknown, Question: How Did You Hear About Us?*, aria-labelledby: source-section
Input ID: unknown, Question: Have you previously worked for Harris?*, aria-labelledby: None
Input ID: unknown, Question: Have you previously worked for Harris?*, aria-labelledby: None
Input ID: unknown, Question: Country*, aria-labelledby: country-section
Input ID: unknown, Question: Country*, aria-labelledby: country-section
Input ID: unknown, Question: Name, aria-labelledby: Name-section
Input ID: unknown, Question: Name, aria-labelledby: Name-section
Input ID: unknown, Question: Address, aria-labelledby: Address-section
Input ID: unknown, Question: Address, aria-labelledby: Address-section
Input ID: unknown, Question: Address, aria-labelledby: Address-section
Input ID: unknown, Question: Address, aria-labelledby: Address-section
Input ID: unknown, Question: Address, aria-labelledby: Address-section
Input I

In [ ]:
async def get_nearest_label_text(element, page):
    # Try explicit <label for="">
    element_id = await element.get_attribute('id')
    if element_id:
        label_elem = await page.query_selector(f'label[for="{element_id}"]')
        if label_elem:
            label_text = await label_elem.text_content()
            if label_text:
                return label_text.strip()
    # Try parent label
    parent_label_handle = await element.evaluate_handle('el => el.closest("label")')
    parent_label = parent_label_handle.as_element() if parent_label_handle else None
    if parent_label:
        label_text = await parent_label.text_content()
        if label_text:
            return label_text.strip()
    # Try nearest formField label
    form_field_label = await element.evaluate('''
        el => {
            let cur = el.parentElement;
            let depth = 0;
            while (cur && depth < 10) {
                if (cur.tagName.toLowerCase() === "div" &&
                    cur.getAttribute("data-automation-id")?.startsWith("formField-")) {
                    const lbl = cur.querySelector("label span, label");
                    if (lbl && lbl.textContent) return lbl.textContent.trim();
                }
                cur = cur.parentElement;
                depth++;
            }
            return null;
        }
    ''')
    if form_field_label:
        return form_field_label.strip()
    # Fallback: None
    return None

async def get_ai_response_for_panel(current_data, panel_elements, client):
    # Construct the list of questions for the panel with complete element info
    form_fields = []
    key_mapping = {}  # Add this to track the mapping

    for el in panel_elements:
        # Create the full key format that AI will use
        full_key = f"[{el['question']}, {el['input_id']}, {el['input_type']}, {el['aria_labelledby']}, {el['input_tag']}]"
        
        form_fields.append({
            "full_key": full_key,  # Add the full key to the form field data
            "question": el['question'],
            "input_id": el['input_id'],
            "input_type": el['input_type'],
            "input_tag": el['input_tag'],
            "aria_labelledby": el['aria_labelledby'],
            "options": el['options'], 
            "placeholder": el.get('placeholder'),
            "required": el.get('required'),
            "role": el.get('role')
        })
        
        # Store mapping from full key to element
        key_mapping[full_key] = el

    prompt = f"""
You are helping fill a job application form.
You are mapping user profile data to a web form.

You are given:
- An Entry from user profile data (JSON)
- A list of form fields from the application panel (including labels, field types, and available options if there is dropdown for the element)

Return a JSON dictionary mapping the EXACT "full_key" values to appropriate values. Use the user data to fill the values. If a field is not relevant, map it to "SKIP".

CRITICAL: You MUST use the EXACT "full_key" value as the key in your response JSON. Do NOT use just the question text.

IMPORTANT RULES:
- For fields with "options" not None:
  - You MUST select ONLY from the list of provided OPTIONS (case-sensitive)
  - If the user data is longer (e.g., "Bachelor of Engineering in Computer Science") and options are shorter (e.g., "BS"), choose the CLOSEST MATCH based on meaning
  - If no match is appropriate, use "SKIP"
- For date fields: Month should be number format (e.g., "01" for January), year should be "YYYY" format
- For date-related fields (e.g. type="spinbutton" or input_id includes "Month" or "Year"):
  - Use "MM" format for months (e.g., "01" for January)
  - Use "YYYY" format for years (e.g., "2022")
  - Match "start date", "end date", "graduation date", etc., with the corresponding data from user profile
- For text fields: Keep responses concise and relevant
- Match options exactly as they appear in the options list (case-sensitive) when options is not None
- After filling the form, if a field for save and continue is present, respond with yes to save the form
- For any question regarding details of the language proficiency, respond with the most appropriate answer based on the user's profile. DO NOT use "SKIP" for these questions.

SPECIAL HANDLING FOR SKILLS/MULTI-VALUE FIELDS:
- For fields related to skills, technologies, competencies, or any field that should contain multiple items:
  - Return an ARRAY of strings instead of a single comma-separated string
  - Each skill/technology should be a separate string in the array
  - Example: ["C#", "TypeScript", "Java", "SQL", "HTML5", "CSS3", "Python"] instead of "C#, TypeScript, Java, SQL, HTML5, CSS3, Python"
- Identify skills fields by keywords in the question like: "skills", "technologies", "competencies", "tools", "programming languages", etc.

Data from User Profile:
{json.dumps(current_data, indent=2)}

Form Fields:
{json.dumps(form_fields, indent=2)}

Example response format:
{{
  "[School or University*, unknown, text, Education-(Optional)-2-panel, input]": "University Name",
  "[Degree*, unknown, button, Education-(Optional)-2-panel, button]": "MS",
  "[Field of Study, unknown, unknown, Education-(Optional)-2-panel, input]": "Computer Science",
  "[Type to Add Skills, unknown, unknown, Skills-section, input]": ["C#", "TypeScript", "Java", "SQL", "HTML5", "CSS3", "Python", ".NET Core", "Angular 2+", "RxJS", "Entity Framework", "React", "Redux", "Bootstrap 4"]
}}

Respond ONLY with a valid JSON object using the exact "full_key" values as keys.
"""
    
    try:
        response = await client.chat.completions.create(
            model="o4-mini",
            messages=[{"role": "user", "content": prompt}]
            # temperature=0.2,
            # max_tokens=500,
        )


        print("Raw Response: ", response)
        content = response.choices[0].message.content.strip()
        print("AI Response Content: ", content)
        # Remove markdown code block formatting if present
        if content.startswith('```json'):
            content = content[7:]  # Remove ```json
        if content.startswith('```'):
            content = content[3:]   # Remove ```
        if content.endswith('```'):
            content = content[:-3]  # Remove closing ```
        content = content.strip()

        print("Cleaned AI Response Content: ", content)
        ai_response = json.loads(content)
        print("Parsed AI Response: ", ai_response)
        return ai_response, key_mapping
    except Exception as e:
        print(f"Error in get_ai_response_for_panel: {e}")
        return {}, {}
    return {},{}

async def get_listbox_options(input_el, page, full_page):
    """Extract options from a listbox by clicking it and reading the <li> elements"""
    print("Starting to get listbox options")
    await input_el.click()
    await asyncio.sleep(1)  # Wait for options to appear
    
    # Look for the opened listbox container with visibility="opened"
    selector = 'div[visibility="opened"]'

    options = []
    listbox_container = await full_page.query_selector(selector)
    
    if listbox_container:
        li_elements = await listbox_container.query_selector_all('li[role="option"]')

        for li in li_elements:
            # Get text content, which will extract from nested elements like <div>
            text = await li.text_content()
            if text and text.strip():
                options.append(text.strip())
            else:
                # Fallback: try to get text from nested div if direct text_content fails
                div_text = await li.query_selector('div')
                if div_text:
                    nested_text = await div_text.text_content()
                    if nested_text and nested_text.strip():
                        options.append(nested_text.strip())

    # Close the listbox after getting options
    await input_el.click()
    await asyncio.sleep(0.5)
    print("Options : ", options)
    return options

async def fill_listbox_element(input_el, page, response, full_page):
    """Fill a listbox/combobox element with the given response"""
    # Click to open the listbox
    await input_el.click()
    await asyncio.sleep(0.5)
    
    # Look for the opened listbox with the specific visibility attribute
    listbox_selectors = [
        'div[visibility="opened"]',
    ]
    
    for selector in listbox_selectors:
        listbox = await full_page.query_selector(selector)
        if listbox:
            li_elements = await listbox.query_selector_all('li')
            for li in li_elements:
                # Get text content from li or nested elements
                text = await li.text_content()
                print(text.lower())
                print(response.lower())
                if text.lower() == response.lower():
                    await li.click()
                    # await asyncio.sleep(0.2)
                    return True
                else:
                    # Fallback: check nested div content
                    div_element = await li.query_selector('div')
                    if div_element:
                        div_text = await div_element.text_content()
                        if div_text and response.lower() in div_text.lower():
                            await li.click()
                            await asyncio.sleep(0.2)
                            return True
            break
    
    # If not found, close the listbox
    # await input_el.keyboard.press('Escape')
    return False
    
async def get_element_options(input_el, page, input_tag, input_type, full_page):
    """Get options for different input types"""
    options = None
    
    if input_tag == "button" or await input_el.get_attribute('role') == 'combobox':
        # Handle listbox/combobox buttons
        aria_haspopup = await input_el.get_attribute('aria-haspopup')
        if aria_haspopup == "listbox":
            options = await get_listbox_options(input_el, page, full_page)
    
    return options

async def fill_element(full_page, input_el, input_id, input_type, input_tag, response, options=None):
    try:
        if response == "SKIP":
            print(f"Skipping input {input_id} as per AI {response}")
            return

        if input_tag == "input" and input_type == "file":
            print(" --DEBUG-- Filling file upload input")
            await input_el.set_input_files([response])
            return

        # TEXT inputs or TEXTAREAs
        if input_tag in ["input", "textarea"] and not await input_el.get_attribute("dir") and input_type not in ["radio", "checkbox", "spinbutton"]:
            print(f"Filling input {input_id} with response: {response}")
            await input_el.fill(response)
            return

        # This is for multiInputContainer elements
        elif input_tag in ["input", "textarea"] and await input_el.get_attribute("dir") and input_type not in ["radio", "checkbox", "spinbutton"]:
            
            # if response is a list
            if isinstance(response, list):
                pass
            else:
                response = [response]      
            
            for res in response:
                print(f"Filling multiInputContainer {input_id} with response: {res}")
                await input_el.click()
                await asyncio.sleep(0.5)  # Wait for focus
                await input_el.fill(res)
                await input_el.press('Enter')  # Submit the input if needed
                await asyncio.sleep(2)  # Wait for any potential UI updates
                # if there are promptOption options, select the first one
                # break
                prompt_options = await full_page.query_selector_all(
                    'div[data-automation-id="promptOption"], div[data-automation-id="promptLeafNode"]'
                )
                # the prompt_options tagName shouldn't be <p (as in paragraph) but <span> or <div>
                print("Before : ", prompt_options)
                # prompt_options = [opt for opt in prompt_options if not await opt.evaluate('(el) => el.tagName.toLowerCase()') == 'p']
                # print("After : ", prompt_options)
                if prompt_options:
                    first_option = prompt_options[0]
                    await first_option.click()
                    print(f"Selected first promptOption: {await first_option.text_content()}")

        if input_type == "checkbox":
            is_checked = await input_el.is_checked()
            should_be_checked = str(response).lower() in ["yes", "true", "1", "check", "selected"] or response is True
            
            if should_be_checked and not is_checked:
                await input_el.click()
            elif not should_be_checked and is_checked:
                await input_el.click()
            return

        # SELECT dropdowns
        elif input_tag == "select":
            # Validate response against options if available
            if options and response not in options:
                print(f"[WARN] Response '{response}' not in select options: {options}")
                return
            await input_el.select_option(label=response)

        # LISTBOX/COMBOBOX buttons
        elif input_tag == "button":
            print(f"Filling listbox {input_id} with response: {response}")
            role = await input_el.get_attribute('role')
            aria_haspopup = await input_el.get_attribute('aria-haspopup')
            if aria_haspopup == "listbox":
                # Validate response against options
                if options and not any(response.lower() in opt.lower() for opt in options):
                    print(f"[WARN] Response '{response}' not found in listbox options: {options}")
                    return
                print("Response found in options, proceeding to fill listbox")
                
                success = await fill_listbox_element(input_el, page, response, full_page)
                if not success:
                    print(f"[WARN] Failed to select '{response}' from listbox")
            else:
                print(f"[WARN] Unhandled button role: {role} or aria-haspopup: {aria_haspopup}")

        # DATE (spinbutton)
        elif input_type == "spinbutton":
            await input_el.fill(response)

        # CHECKBOX or RADIO
        elif input_type in ["checkbox", "radio"]:

            response = str(response).lower() if isinstance(response, str) else response

            if response.lower() in ["yes", "true", "1", "check", "selected"]:
                is_checked = await input_el.is_checked()
                if not is_checked:
                    await input_el.click()

        else:
            print(f"[WARN] Unhandled input type/tag: {input_type} / {input_tag}")

    except Exception as e:
        print(f"[ERROR] fill_element failed for {input_tag} / {input_type}: {e}")

async def handle_section_with_add(page, full_page, section_type):

    user_data = await load_user_profile()

    if section_type == "experience":
        work_experiences = user_data.get("work_experience", [])
    if section_type == "education":
        work_experiences = user_data.get("education", [])
    if section_type == "language":
        work_experiences = user_data.get("fluent_languages", [])
    
    # get_user_infor(user_profile, aria_labelledby) -> return user_data_for_section

    print(f"Found {len(work_experiences)} work experience entries")

    for i, experience_data in enumerate(work_experiences):
        print(f"\n=== Filling work experience {i + 1} ===")

        add_btn = await page.query_selector('button[data-automation-id="add-button"]')
        if add_btn:
            await add_btn.click()
            await asyncio.sleep(2)

        inputs = await page.query_selector_all('input, button, textarea, select')
        panel_elements = []

        previous_question = None

        for input_el in inputs:
            input_id = await input_el.get_attribute('data-automation-id') or 'unknown'
            if input_id in ["pageFooterBackButton", "pageFooterNextButton", "backToJobPosting"]:
                continue

            group_label, aria_labelledby = await get_group_label_and_aria(input_el, page)
            question = await get_nearest_label_text(input_el, page)

            if question == previous_question and not input_id == "file-upload-input-ref":
                print(f"Skipping duplicate question at index {i}: '{question}'")
                continue

            # print("DEBUG -- ", question, input_id, aria_labelledby)

            input_type = await input_el.get_attribute('type') or 'unknown'
            role = await input_el.get_attribute('role')
            placeholder = await input_el.get_attribute('placeholder')
            required = await input_el.get_attribute('required')

            if role == "spinbutton":
                input_type = "spinbutton"

            tag_name = await input_el.evaluate("(el) => el.tagName.toLowerCase()")
            if tag_name and tag_name.lower() == 'textarea':
                input_type = 'textarea'
            input_tag = tag_name
            
            # Get options for all relevant input types
            options = await get_element_options(input_el, page, input_tag, input_type, full_page)

            if aria_labelledby and f'{i + 1}-panel' in aria_labelledby:
                panel_elements.append({
                    'question': question or 'UNLABELED',
                    'aria_labelledby': aria_labelledby,
                    'input_type': input_type,
                    'input_tag': input_tag,
                    'input_id': input_id,
                    'options': options,  
                    'placeholder': placeholder,
                    'required': required,
                    'role': role,
                    'input_el': input_el
                })

            previous_question = question

        if panel_elements:
            print(f"Panel Elements Count: {len(panel_elements)}")
            for field in panel_elements:
                input_type = field['input_type']
                options = field['options'] if field['options'] else 'None'
                print(f"Element: {field['question']}, Type: {input_type}, Options: {options}")

        # Get AI response with complete element information
        ai_values, key_mapping = await get_ai_response_for_panel(experience_data, panel_elements, client)
        print("AI Response:", ai_values)

        # Fill all elements with validation
        for field in panel_elements:
            # Create the same key format used by AI
            full_key = f"[{field['question']}, {field['input_id']}, {field['input_type']}, {field['aria_labelledby']}, {field['input_tag']}]"
            response = ai_values.get(full_key, 'SKIP')
            
            print(f"Filling '{field['question']}' with: {response}")
            print(f"Using key: {full_key}")
            print(f"Filling element: {field['input_tag']} | {field['input_type']} | Question: {field['question']} | Response: {response}")

            await fill_element(
                full_page,
                field['input_el'], 
                field['input_id'], 
                field['input_type'], 
                field['input_tag'], 
                response,
                field['options']
            )
            await asyncio.sleep(0.5)  # Wait for any potential UI updates

main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')

INPUT_SELECTOR = 'button, input, select, textarea, [role="button"]' # Example selector

inputs = await main_page.query_selector_all(INPUT_SELECTOR)

i = 0

prev_answered_question = None
prev_type = None

for input_el in inputs:
    input_id = await input_el.get_attribute('data-automation-id') or await input_el.get_attribute('aria-haspopup') or 'unknown'
    group_label, aria_labelledby = await get_group_label_and_aria(input_el, page)
    question = group_label or 'UNLABELED'
    if input_id == "pageFooterBackButton" or input_id == "backToJobPosting":
        continue
    print()
    print(f"Input ID: {input_id}, Question: {question}, aria-labelledby: {aria_labelledby or 'None'}")
    print(f"Previous question : {prev_answered_question}, Previous type: {prev_type}")
    if question != 'UNLABELED' and (question == prev_answered_question and prev_type == input_type) and not input_id == "file-upload-input-ref":
        print(f" Skipping duplicate question at index {i}: '{question}'")
        i += 1
        continue

    if input_id == "add-button":
        print("Processing add button...")
        main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')
        work_experience_section = await main_page.query_selector(f'div[aria-labelledby="{aria_labelledby}"]')
        if any(keyword in aria_labelledby.lower() for keyword in ["work", "experience", "history"]):
            print("Processing Work Experience")
            await handle_section_with_add(work_experience_section, page, "experience")
        if any(keyword in aria_labelledby.lower() for keyword in ["education", "school", "college"]):
            print("Processing Education")
            await handle_section_with_add(work_experience_section, page, "education")
        if "language" in aria_labelledby.lower():
            print("Processing Language")
            await handle_section_with_add(work_experience_section, page, "language")

    elif input_id == "pageFooterNextButton":
        await input_el.click()

    # Process non-add-button elements
    else:
        user_data = await load_user_profile()
        main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')
        if aria_labelledby:
            section = await main_page.query_selector(f'div[aria-labelledby="{aria_labelledby}"]')
        else:
            section = main_page
        input_type = await input_el.get_attribute('type') or 'unknown'
        role = await input_el.get_attribute('role')
        placeholder = await input_el.get_attribute('placeholder')
        required = await input_el.get_attribute('required')

        if role == "spinbutton":
            input_type = "spinbutton"

        tag_name = await input_el.evaluate("(el) => el.tagName.toLowerCase()")
        if tag_name and tag_name.lower() == 'textarea':
            input_type = 'textarea'
        input_tag = tag_name
        
        # Get options for all relevant input types
        options = await get_element_options(input_el, section, input_tag, input_type, page)
        
        # Get the nearest question/label for this element
        element_question = await get_nearest_label_text(input_el, section)

        print(f"Processing general form element: {element_question}")
        
        # Create element info for AI processing
        element_info = {
            'question': element_question or 'UNLABELED',
            'aria_labelledby': aria_labelledby,
            'input_type': input_type,
            'input_tag': input_tag,
            'input_id': input_id,
            'options': options,
            'placeholder': placeholder,
            'required': required,
            'role': role
        }
        
        print(element_info)

        # # Get AI response using general user data
        # full_key = f"[{element_info['question']}, {element_info['input_id']}, {element_info['input_type']}, {element_info['aria_labelledby']}, {element_info['input_tag']}]"
        # ai_values, _ = await get_ai_response_for_panel(user_data, [element_info], client)
        # response = ai_values.get(full_key, 'SKIP')
        
        # print(f"AI response for general field '{element_question}': {response}")

        # await fill_element(
        #     page,
        #     input_el, 
        #     input_id, 
        #     input_type, 
        #     input_tag, 
        #     response,
        #     options
        # )

    if question != 'UNLABELED':
        prev_answered_question = question

    inputs = await main_page.query_selector_all(INPUT_SELECTOR)

    i += 1


Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Previous question : None
Processing general form element: None
{'question': 'UNLABELED', 'aria_labelledby': None, 'input_type': 'unknown', 'input_tag': 'button', 'input_id': 'unknown', 'options': None, 'placeholder': None, 'required': None, 'role': None}

Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Previous question : None
Processing general form element: None
{'question': 'UNLABELED', 'aria_labelledby': None, 'input_type': 'unknown', 'input_tag': 'button', 'input_id': 'unknown', 'options': None, 'placeholder': None, 'required': None, 'role': None}

Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Previous question : None
Processing general form element: None
{'question': 'UNLABELED', 'aria_labelledby': None, 'input_type': 'unknown', 'input_tag': 'button', 'input_id': 'unknown', 'options': None, 'placeholder': None, 'required': None, 'role': None}

Input ID: unknown, Question: UNLABELED, aria-

# Application Questions

In [ ]:
await asyncio.sleep(10)  # Wait for any potential UI updates

main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')

inputs = await main_page.query_selector_all('input, button, textarea, select')

for input_el in inputs:
    input_id = await input_el.get_attribute('data-automation-id') or await input_el.get_attribute('aria-haspopup') or 'unknown'
    group_label, aria_labelledby = await get_group_label_and_aria(input_el, page)
    question = group_label or 'UNLABELED'
    
    if input_id == "pageFooterBackButton" or input_id == "backToJobPosting":
        continue

    print(f"Input ID: {input_id}, Question: {question}, aria-labelledby: {aria_labelledby or 'None'}")

Input ID: listbox, Question: How Did You Hear About Us?*, aria-labelledby: source-section
Input ID: unknown, Question: How Did You Hear About Us?*, aria-labelledby: source-section
Input ID: unknown, Question: Have you previously worked for Harris?*, aria-labelledby: None
Input ID: unknown, Question: Have you previously worked for Harris?*, aria-labelledby: None
Input ID: listbox, Question: Country*, aria-labelledby: country-section
Input ID: unknown, Question: Country*, aria-labelledby: country-section
Input ID: unknown, Question: Name, aria-labelledby: Name-section
Input ID: unknown, Question: Name, aria-labelledby: Name-section
Input ID: unknown, Question: Address, aria-labelledby: Address-section
Input ID: unknown, Question: Address, aria-labelledby: Address-section
Input ID: listbox, Question: Address, aria-labelledby: Address-section
Input ID: unknown, Question: Address, aria-labelledby: Address-section
Input ID: unknown, Question: Address, aria-labelledby: Address-section
Input I

In [ ]:
async def get_nearest_label_text(element, page):
    # Try explicit <label for="">
    element_id = await element.get_attribute('id')
    print(f"Element ID: {element_id}")
    if element_id != "unknown":
        label_elem = await page.query_selector(f'label[for="{element_id}"]')
        if label_elem:
            label_text = await label_elem.text_content()
            if label_text:
                return label_text.strip()
    # Try parent label
    parent_label_handle = await element.evaluate_handle('el => el.closest("label")')
    parent_label = parent_label_handle.as_element() if parent_label_handle else None
    if parent_label:
        label_text = await parent_label.text_content()
        if label_text:
            return label_text.strip()
    # Try nearest formField label
    form_field_label = await element.evaluate('''
        el => {
            let cur = el.parentElement;
            let depth = 0;
            while (cur && depth < 10) {
                if (cur.tagName.toLowerCase() === "div" &&
                    cur.getAttribute("data-automation-id")?.startsWith("formField-")) {
                    const lbl = cur.querySelector("label span, label");
                    if (lbl && lbl.textContent) return lbl.textContent.trim();
                }
                cur = cur.parentElement;
                depth++;
            }
            return null;
        }
    ''')
    if form_field_label:
        return form_field_label.strip()
    # Fallback: None
    return None

async def get_ai_response_for_panel(current_data, panel_elements, client):
    # Construct the list of questions for the panel with complete element info
    form_fields = []
    key_mapping = {}  # Add this to track the mapping

    for el in panel_elements:
        # Create the full key format that AI will use
        full_key = f"[{el['question']}, {el['input_id']}, {el['input_type']}, {el['aria_labelledby']}, {el['input_tag']}]"
        
        form_fields.append({
            "full_key": full_key,  # Add the full key to the form field data
            "question": el['question'],
            "input_id": el['input_id'],
            "input_type": el['input_type'],
            "input_tag": el['input_tag'],
            "aria_labelledby": el['aria_labelledby'],
            "options": el['options'], 
            "placeholder": el.get('placeholder'),
            "required": el.get('required'),
            "role": el.get('role')
        })
        
        # Store mapping from full key to element
        key_mapping[full_key] = el

    prompt = f"""
You are helping fill a job application form.
You are mapping user profile data to a web form.

You are given:
- An Entry from user profile data (JSON)
- A list of form fields from the application panel (including labels, field types, and available options if there is dropdown for the element)

Return a JSON dictionary mapping the EXACT "full_key" values to appropriate values. Use the user data to fill the values. If a field is not relevant, map it to "SKIP".

CRITICAL: You MUST use the EXACT "full_key" value as the key in your response JSON. Do NOT use just the question text.

IMPORTANT RULES:
- For fields with "options" not None:
  - You MUST select ONLY from the list of provided OPTIONS (case-sensitive)
  - If the user data is longer (e.g., "Bachelor of Engineering in Computer Science") and options are shorter (e.g., "BS"), choose the CLOSEST MATCH based on meaning
  - If no match is appropriate, use "SKIP"
- For date fields: Month should be number format (e.g., "01" for January), year should be "YYYY" format
- For date-related fields (e.g. type="spinbutton" or input_id includes "Month" or "Year"):
  - Use "MM" format for months (e.g., "01" for January)
  - Use "YYYY" format for years (e.g., "2022")
  - Match "start date", "end date", "graduation date", etc., with the corresponding data from user profile
- For text fields: Keep responses concise and relevant
- Match options exactly as they appear in the options list (case-sensitive) when options is not None
- After filling the form, if a field for save and continue is present, respond with yes to save the form
- Incase of Yes or No questions in listbox in application form, respond with the most appropriate answer and do not SKIP.

SPECIAL HANDLING FOR SKILLS/MULTI-VALUE FIELDS:
- For fields related to skills, technologies, competencies, or any field that should contain multiple items:
  - Return an ARRAY of strings instead of a single comma-separated string
  - Each skill/technology should be a separate string in the array
  - Example: ["C#", "TypeScript", "Java", "SQL", "HTML5", "CSS3", "Python"] instead of "C#, TypeScript, Java, SQL, HTML5, CSS3, Python"
- Identify skills fields by keywords in the question like: "skills", "technologies", "competencies", "tools", "programming languages", etc.

Data from User Profile:
{json.dumps(current_data, indent=2)}

Form Fields:
{json.dumps(form_fields, indent=2)}

Example response format:
{{
  "[School or University*, unknown, text, Education-(Optional)-2-panel, input]": "University Name",
  "[Degree*, unknown, button, Education-(Optional)-2-panel, button]": "MS",
  "[Field of Study, unknown, unknown, Education-(Optional)-2-panel, input]": "Computer Science",
  "[Type to Add Skills, unknown, unknown, Skills-section, input]": ["C#", "TypeScript", "Java", "SQL", "HTML5", "CSS3", "Python", ".NET Core", "Angular 2+", "RxJS", "Entity Framework", "React", "Redux", "Bootstrap 4"]
}}

Respond ONLY with a valid JSON object using the exact "full_key" values as keys.
"""
    
    try:
        response = await client.chat.completions.create(
            model="o4-mini",
            messages=[{"role": "user", "content": prompt}]
            # temperature=0.2,
            # max_tokens=500,
        )
        print("Raw Response: ", response)
        content = response.choices[0].message.content.strip()
        print("AI Response Content: ", content)
        ai_response = json.loads(content)
        print("Parsed AI Response: ", ai_response)
        return ai_response, key_mapping
    except Exception as e:
        print(f"Error in get_ai_response_for_panel: {e}")
        return {}, {}
    return {},{}

async def get_listbox_options(input_el, page, full_page):
    """Extract options from a listbox by clicking it and reading the <li> elements"""
    print("Starting to get listbox options")
    await input_el.click()
    await asyncio.sleep(1)  # Wait for options to appear
    
    # Look for the opened listbox container with visibility="opened"
    selector = 'div[visibility="opened"]'

    options = []
    listbox_container = await full_page.query_selector(selector)
    
    if listbox_container:
        li_elements = await listbox_container.query_selector_all('li[role="option"]')

        for li in li_elements:
            # Get text content, which will extract from nested elements like <div>
            text = await li.text_content()
            if text and text.strip():
                options.append(text.strip())
            else:
                # Fallback: try to get text from nested div if direct text_content fails
                div_text = await li.query_selector('div')
                if div_text:
                    nested_text = await div_text.text_content()
                    if nested_text and nested_text.strip():
                        options.append(nested_text.strip())

    # Close the listbox after getting options
    await input_el.click()
    await asyncio.sleep(0.5)
    print("Options : ", options)
    return options

async def fill_listbox_element(input_el, page, response, full_page):
    """Fill a listbox/combobox element with the given response"""
    # Click to open the listbox
    await input_el.click()
    await asyncio.sleep(0.5)
    
    # Look for the opened listbox with the specific visibility attribute
    listbox_selectors = [
        'div[visibility="opened"]',
    ]
    
    for selector in listbox_selectors:
        listbox = await full_page.query_selector(selector)
        if listbox:
            li_elements = await listbox.query_selector_all('li')
            for li in li_elements:
                # Get text content from li or nested elements
                text = await li.text_content()
                print(text.lower())
                print(response.lower())
                if text.lower() == response.lower():
                    await li.click()
                    # await asyncio.sleep(0.2)
                    return True
                else:
                    # Fallback: check nested div content
                    div_element = await li.query_selector('div')
                    if div_element:
                        div_text = await div_element.text_content()
                        if div_text and response.lower() in div_text.lower():
                            await li.click()
                            await asyncio.sleep(0.2)
                            return True
            break
    
    # If not found, close the listbox
    # await input_el.keyboard.press('Escape')
    return False
    
async def get_element_options(input_el, page, input_tag, input_type, full_page):
    """Get options for different input types"""
    options = None
    
    if input_tag == "button" or await input_el.get_attribute('role') == 'combobox':
        # Handle listbox/combobox buttons
        aria_haspopup = await input_el.get_attribute('aria-haspopup')
        if aria_haspopup == "listbox":
            options = await get_listbox_options(input_el, page, full_page)
    
    return options

async def fill_element(full_page, input_el, input_id, input_type, input_tag, response, options=None):
    try:
        if response == "SKIP":
            print(f"Skipping input {input_id} as per AI {response}")
            return

        if input_tag == "input" and input_type == "file":
            print(" --DEBUG-- Filling file upload input")
            await input_el.set_input_files([response])
            print("File input filled successfully")
            return

        # TEXT inputs or TEXTAREAs
        if input_tag in ["input", "textarea"] and not await input_el.get_attribute("dir") and input_type not in ["radio", "checkbox", "spinbutton"]:
            print(f"Filling input {input_id} with response: {response}")
            await input_el.fill(response)
            print(f"Input filled successfully")
            return

        # This is for multiInputContainer elements
        elif input_tag in ["input", "textarea"] and await input_el.get_attribute("dir") and input_type not in ["radio", "checkbox", "spinbutton"]:
            
            # if response is a list
            if isinstance(response, list):
                pass
            else:
                response = [response]      
            
            for res in response:
                print(f"Filling multiInputContainer {input_id} with response: {res}")
                await input_el.click()
                await asyncio.sleep(0.5)  # Wait for focus
                await input_el.fill(res)
                await input_el.press('Enter')  # Submit the input if needed
                await asyncio.sleep(1)  # Wait for any potential UI updates
                # if there are promptOption options, select the first one
                # break
                prompt_options = await full_page.query_selector_all(
                    'div[data-automation-id="promptOption"], div[data-automation-id="promptLeafNode"]'
                )
                # the prompt_options tagName shouldn't be <p (as in paragraph) but <span> or <div>
                print("Before : ", prompt_options)
                # prompt_options = [opt for opt in prompt_options if not await opt.evaluate('(el) => el.tagName.toLowerCase()') == 'p']
                # print("After : ", prompt_options)
                if prompt_options:
                    first_option = prompt_options[0]
                    await first_option.click()
                    print(f"Selected first promptOption: {await first_option.text_content()}")
                
            print("MultiInputContainer filled successfully")

        elif input_type == "checkbox":
            is_checked = await input_el.is_checked()
            should_be_checked = str(response).lower() in ["yes", "true", "1", "check", "selected"] or response is True
            
            if should_be_checked and not is_checked:
                await input_el.click()
            elif not should_be_checked and is_checked:
                await input_el.click()
            return

        # SELECT dropdowns
        elif input_tag == "select":
            # Validate response against options if available
            if options and response not in options:
                print(f"[WARN] Response '{response}' not in select options: {options}")
                return
            await input_el.select_option(label=response)
            print(f"Select dropdown {input_id} filled successfully with response: {response}")

        # LISTBOX/COMBOBOX buttons
        elif input_tag == "button":
            print(f"Filling listbox {input_id} with response: {response}")
            role = await input_el.get_attribute('role')
            aria_haspopup = await input_el.get_attribute('aria-haspopup')
            if aria_haspopup == "listbox":
                # Validate response against options
                if options and not any(response.lower() in opt.lower() for opt in options):
                    print(f"[WARN] Response '{response}' not found in listbox options: {options}")
                    return
                print("Response found in options, proceeding to fill listbox")
                
                success = await fill_listbox_element(input_el, page, response, full_page)
                if not success:
                    print(f"[WARN] Failed to select '{response}' from listbox")
            else:
                print(f"[WARN] Unhandled button role: {role} or aria-haspopup: {aria_haspopup}")
            print(f"Listbox {input_id} filled successfully with response: {response}")

        # DATE (spinbutton)
        elif input_type == "spinbutton":
            await input_el.fill(response)
            print(f"Spinbutton {input_id} filled successfully with response: {response}")

        # CHECKBOX or RADIO
        elif input_type in ["checkbox", "radio"]:
            if response.lower() in ["yes", "true", "1", "check", "selected"]:
                is_checked = await input_el.is_checked()
                if not is_checked:
                    await input_el.click()
            print(f"Checkbox/Radio {input_id} filled successfully with response: {response}")

        else:
            print(f"[WARN] Unhandled input type/tag: {input_type} / {input_tag}")
            print(f"Input ID: {input_id}, Type: {input_type}, Tag: {input_tag}, Response: {response}")

    except Exception as e:
        print(f"[ERROR] fill_element failed for {input_tag} / {input_type}: {e}")

asyncio.sleep(5)  # Wait for any potential UI updates

main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')

INPUT_SELECTOR = 'button, input, select, textarea, [role="button"]' # Example selector

inputs = await main_page.query_selector_all(INPUT_SELECTOR)

i = 0

prev_answered_question = None

while i < len(inputs):

    input_el = inputs[i]

    # Your existing logic to process the element
    input_id = await input_el.get_attribute('data-automation-id') or await input_el.get_attribute('aria-haspopup') or 'unknown'
    group_label, aria_labelledby = await get_group_label_and_aria(input_el, page)
    question = group_label or 'UNLABELED'

    if question != 'UNLABELED' and question == prev_answered_question:
        print(f"⏩ Skipping duplicate question at index {i}: '{question}'")
        i += 1
        continue

    if input_id == "pageFooterBackButton" or input_id == "backToJobPosting":
        # We just want to skip this element and move to the next one
        i += 1
        continue

    print(f"Input ID: {input_id}, Question: {question}, aria-labelledby: {aria_labelledby or 'None'}")

    if input_id == "pageFooterNextButton":
        print("Clicking Next button")
        await input_el.click()

    # Process non-add-button elements
    else:
        user_data = await load_user_profile()
        main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')
        section = await main_page.query_selector(f'div[aria-labelledby="{aria_labelledby}"]')

        input_type = await input_el.get_attribute('type') or 'unknown'
        role = await input_el.get_attribute('role')
        placeholder = await input_el.get_attribute('placeholder')
        required = await input_el.get_attribute('required')

        if role == "spinbutton":
            input_type = "spinbutton"

        tag_name = await input_el.evaluate("(el) => el.tagName.toLowerCase()")
        if tag_name and tag_name.lower() == 'textarea':
            input_type = 'textarea'
        input_tag = tag_name
        
        # Get options for all relevant input types
        options = await get_element_options(input_el, section, input_tag, input_type, page)
        
        # Get the nearest question/label for this element
        # element_question = await get_nearest_label_text(input_el, section)

        print(f"Processing general form element: {question}")
        
        # Create element info for AI processing
        element_info = {
            'question': question or 'UNLABELED',
            'aria_labelledby': aria_labelledby,
            'input_type': input_type,
            'input_tag': input_tag,
            'input_id': input_id,
            'options': options,
            'placeholder': placeholder,
            'required': required,
            'role': role
        }
        
        print(element_info)

        # # Get AI response using general user data
        full_key = f"[{element_info['question']}, {element_info['input_id']}, {element_info['input_type']}, {element_info['aria_labelledby']}, {element_info['input_tag']}]"
        ai_values, _ = await get_ai_response_for_panel(user_data, [element_info], client)
        response = ai_values.get(full_key, 'SKIP')
        
        print(f"AI response for general field '{question}': {response}")

        await fill_element(
            page,
            input_el, 
            input_id, 
            input_type, 
            input_tag, 
            response,
            options
        )

        if question != 'UNLABELED':
            prev_answered_question = question

    inputs = await main_page.query_selector_all(INPUT_SELECTOR)

    i += 1

/var/folders/bc/7dmh10j96rscf81jsj7gbw8h0000gn/T/ipykernel_6366/1778351338.py:328: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(5)  # Wait for any potential UI updates


Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Processing general form element: UNLABELED
{'question': 'UNLABELED', 'aria_labelledby': None, 'input_type': 'unknown', 'input_tag': 'button', 'input_id': 'unknown', 'options': None, 'placeholder': None, 'required': None, 'role': None}
Raw Response:  ChatCompletion(id='chatcmpl-C14gPgElYcOE8y6ee9A3TRcsvW5AE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "[UNLABELED, unknown, unknown, None, button]": "yes"\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1754372365, model='o4-mini-2025-04-16', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=231, prompt_tokens=2248, total_tokens=2479, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=192, rejected_prediction_tokens=0), prompt_t

# Voluntary Disclosures

In [ ]:
await asyncio.sleep(10)  # Wait for any potential UI updates

async def get_nearest_label_text(element, page):
    """Get the nearest label text for a form element by checking multiple DOM patterns"""
    try:
        # Try explicit <label for="">
        element_id = await element.get_attribute('id')
        if element_id and element_id != "unknown":
            label_elem = await page.query_selector(f'label[for="{element_id}"]')
            if label_elem:
                label_text = await label_elem.text_content()
                if label_text:
                    return label_text.replace('*', '').strip()
        
        # Try parent label (element wrapped in label)
        parent_label_handle = await element.evaluate_handle('el => el.closest("label")')
        parent_label = parent_label_handle.as_element() if parent_label_handle else None
        if parent_label:
            label_text = await parent_label.text_content()
            if label_text:
                return label_text.replace('*', '').strip()
        
        # Try nearest formField label in parent containers
        form_field_label = await element.evaluate('''
            el => {
                let cur = el.parentElement;
                let depth = 0;
                while (cur && depth < 10) {
                    if (cur.tagName.toLowerCase() === "div" &&
                        cur.getAttribute("data-automation-id")?.startsWith("formField-")) {
                        const lbl = cur.querySelector("label span, label");
                        if (lbl && lbl.textContent) return lbl.textContent.trim();
                    }
                    cur = cur.parentElement;
                    depth++;
                }
                return null;
            }
        ''')
        if form_field_label:
            return form_field_label.replace('*', '').strip()
        
        # Try aria-labelledby attribute
        aria_labelledby = await element.get_attribute('aria-labelledby')
        if aria_labelledby:
            label_element = await page.query_selector(f'#{aria_labelledby}')
            if label_element:
                label_text = await label_element.text_content()
                if label_text:
                    return label_text.replace('*', '').strip()
        
        # Try aria-label attribute
        aria_label = await element.get_attribute('aria-label')
        if aria_label:
            return aria_label.replace('*', '').strip()
        
        # Try placeholder as fallback
        placeholder = await element.get_attribute('placeholder')
        if placeholder:
            return placeholder.replace('*', '').strip()
        
        # Try searching for nearby text nodes or labels
        nearby_label = await element.evaluate('''
            el => {
                // Check previous sibling elements for labels
                let prev = el.previousElementSibling;
                while (prev) {
                    if (prev.tagName.toLowerCase() === 'label' || prev.textContent.trim()) {
                        return prev.textContent.trim();
                    }
                    prev = prev.previousElementSibling;
                }
                
                // Check parent's previous siblings
                let parent = el.parentElement;
                if (parent) {
                    let prevParent = parent.previousElementSibling;
                    while (prevParent) {
                        const label = prevParent.querySelector('label');
                        if (label && label.textContent) {
                            return label.textContent.trim();
                        }
                        if (prevParent.textContent.trim()) {
                            return prevParent.textContent.trim();
                        }
                        prevParent = prevParent.previousElementSibling;
                    }
                }
                
                return null;
            }
        ''')
        if nearby_label:
            return nearby_label.replace('*', '').strip()
        
        return None
        
    except Exception as e:
        print(f"Error getting label for element: {e}")
        return None

main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')

inputs = await main_page.query_selector_all('input, button, textarea, select')

for input_el in inputs:
    input_id = await input_el.get_attribute('data-automation-id') or await input_el.get_attribute('aria-haspopup') or 'unknown'
    group_label, aria_labelledby = await get_group_label_and_aria(input_el, page)
    question = group_label or 'UNLABELED'

    question = await get_nearest_label_text(input_el, page) or question

    if input_id == "pageFooterBackButton" or input_id == "backToJobPosting":
        continue

    print(f"Input ID: {input_id}, Question: {question}, aria-labelledby: {aria_labelledby or 'None'}")

Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Input ID: unknown, Question: Work Experience 1, aria-labelledby: Work-Experience-1-panel
Input ID: unknown, Question: Job Title, aria-labelledby: Work-Experience-1-panel
Input ID: unknown, Question: Company, aria-labelledby: Work-Experience-1-panel
Input ID: unknown, Question: Location, aria-labelledby: Work-Experience-1-panel
Input ID: unknown, Question: I currently work here, aria-labelledby: Work-Experience-1-panel
Input ID: dateSectionMonth-input, Question: From, aria-labelledby: Work-Experience

In [ ]:
from datetime import date

async def get_nearest_label_text(element, page):
    # Try explicit <label for="">
    element_id = await element.get_attribute('id')
    print(f"Element ID: {element_id}")
    if element_id != "unknown":
        label_elem = await page.query_selector(f'label[for="{element_id}"]')
        if label_elem:
            label_text = await label_elem.text_content()
            if label_text:
                return label_text.strip()
    # Try parent label
    parent_label_handle = await element.evaluate_handle('el => el.closest("label")')
    parent_label = parent_label_handle.as_element() if parent_label_handle else None
    if parent_label:
        label_text = await parent_label.text_content()
        if label_text:
            return label_text.strip()
    # Try nearest formField label
    form_field_label = await element.evaluate('''
        el => {
            let cur = el.parentElement;
            let depth = 0;
            while (cur && depth < 10) {
                if (cur.tagName.toLowerCase() === "div" &&
                    cur.getAttribute("data-automation-id")?.startsWith("formField-")) {
                    const lbl = cur.querySelector("label span, label");
                    if (lbl && lbl.textContent) return lbl.textContent.trim();
                }
                cur = cur.parentElement;
                depth++;
            }
            return null;
        }
    ''')
    if form_field_label:
        return form_field_label.strip()
    # Fallback: None
    return None

async def get_ai_response_for_panel(current_data, panel_elements, client):
    # Construct the list of questions for the panel with complete element info
    form_fields = []
    key_mapping = {}  # Add this to track the mapping

    for el in panel_elements:
        # Create the full key format that AI will use
        full_key = f"[{el['question']}, {el['input_id']}, {el['input_type']}, {el['aria_labelledby']}, {el['input_tag']}]"
        
        form_fields.append({
            "full_key": full_key,  # Add the full key to the form field data
            "question": el['question'],
            "input_id": el['input_id'],
            "input_type": el['input_type'],
            "input_tag": el['input_tag'],
            "aria_labelledby": el['aria_labelledby'],
            "options": el['options'], 
            "placeholder": el.get('placeholder'),
            "required": el.get('required'),
            "role": el.get('role')
        })
        
        # Store mapping from full key to element
        key_mapping[full_key] = el

    prompt = f"""
You are helping fill a job application form.
You are mapping user profile data to a web form.

You are given:
- An Entry from user profile data (JSON)
- A list of form fields from the application panel (including labels, field types, and available options if there is dropdown for the element)

Return a JSON dictionary mapping the EXACT "full_key" values to appropriate values. Use the user data to fill the values. If a field is not relevant, map it to "SKIP".

CRITICAL: You MUST use the EXACT "full_key" value as the key in your response JSON. Do NOT use just the question text.

IMPORTANT RULES:
- For fields with "options" not None:
  - You MUST select ONLY from the list of provided OPTIONS (case-sensitive)
  - If the user data is longer (e.g., "Bachelor of Engineering in Computer Science") and options are shorter (e.g., "BS"), choose the CLOSEST MATCH based on meaning
  - If no match is appropriate, use "SKIP"
- For date fields: Month should be number format (e.g., "01" for January), year should be "YYYY" format
- For date-related fields (e.g. type="spinbutton" or input_id includes "Month" or "Year"):
  - Use "MM" format for months (e.g., "01" for January)
  - Use "YYYY" format for years (e.g., "2022")
  - Match "start date", "end date", "graduation date", etc., with the corresponding data from user profile
- Make sure not to skip voluntary disclosure questions about gender, ethnicity, disability, and veteran status and other similar questions
- For text fields: Keep responses concise and relevant
- Match options exactly as they appear in the options list (case-sensitive) when options is not None
- After filling the form, if a field for save and continue is present, respond with yes to save the form

SPECIAL HANDLING FOR SKILLS/MULTI-VALUE FIELDS:
- For fields related to skills, technologies, competencies, or any field that should contain multiple items:
  - Return an ARRAY of strings instead of a single comma-separated string
  - Each skill/technology should be a separate string in the array
  - Example: ["C#", "TypeScript", "Java", "SQL", "HTML5", "CSS3", "Python"] instead of "C#, TypeScript, Java, SQL, HTML5, CSS3, Python"
- Identify skills fields by keywords in the question like: "skills", "technologies", "competencies", "tools", "programming languages", etc.

Data from User Profile:
{json.dumps(current_data, indent=2)}

Form Fields:
{json.dumps(form_fields, indent=2)}

Example response format:
{{
  "[School or University*, unknown, text, Education-(Optional)-2-panel, input]": "University Name",
  "[Degree*, unknown, button, Education-(Optional)-2-panel, button]": "MS",
  "[Field of Study, unknown, unknown, Education-(Optional)-2-panel, input]": "Computer Science",
  "[Type to Add Skills, unknown, unknown, Skills-section, input]": ["C#", "TypeScript", "Java", "SQL", "HTML5", "CSS3", "Python", ".NET Core", "Angular 2+", "RxJS", "Entity Framework", "React", "Redux", "Bootstrap 4"]
}}

Respond ONLY with a valid JSON object using the exact "full_key" values as keys.
"""
    
    try:
        response = await client.chat.completions.create(
            model="o4-mini",
            messages=[{"role": "user", "content": prompt}]
            # temperature=0.2,
            # max_tokens=500,
        )
        print("Raw Response: ", response)
        content = response.choices[0].message.content.strip()
        print("AI Response Content: ", content)
        ai_response = json.loads(content)
        print("Parsed AI Response: ", ai_response)
        return ai_response, key_mapping
    except Exception as e:
        print(f"Error in get_ai_response_for_panel: {e}")
        return {}, {}
    return {},{}

async def get_listbox_options(input_el, page, full_page):
    """Extract options from a listbox by clicking it and reading the <li> elements"""
    print("Starting to get listbox options")
    await input_el.click()
    await asyncio.sleep(1)  # Wait for options to appear
    
    # Look for the opened listbox container with visibility="opened"
    selector = 'div[visibility="opened"]'

    options = []
    listbox_container = await full_page.query_selector(selector)
    
    if listbox_container:
        li_elements = await listbox_container.query_selector_all('li[role="option"]')

        for li in li_elements:
            # Get text content, which will extract from nested elements like <div>
            text = await li.text_content()
            if text and text.strip():
                options.append(text.strip())
            else:
                # Fallback: try to get text from nested div if direct text_content fails
                div_text = await li.query_selector('div')
                if div_text:
                    nested_text = await div_text.text_content()
                    if nested_text and nested_text.strip():
                        options.append(nested_text.strip())

    # Close the listbox after getting options
    await input_el.click()
    await asyncio.sleep(0.5)
    print("Options : ", options)
    return options

async def fill_listbox_element(input_el, page, response, full_page):
    """Fill a listbox/combobox element with the given response"""
    # Click to open the listbox
    await input_el.click()
    await asyncio.sleep(0.5)
    
    # Look for the opened listbox with the specific visibility attribute
    listbox_selectors = [
        'div[visibility="opened"]',
    ]
    
    for selector in listbox_selectors:
        listbox = await full_page.query_selector(selector)
        if listbox:
            li_elements = await listbox.query_selector_all('li')
            for li in li_elements:
                # Get text content from li or nested elements
                text = await li.text_content()
                print(text.lower())
                print(response.lower())
                if text.lower() == response.lower():
                    await li.click()
                    # await asyncio.sleep(0.2)
                    return True
                else:
                    # Fallback: check nested div content
                    div_element = await li.query_selector('div')
                    if div_element:
                        div_text = await div_element.text_content()
                        if div_text and response.lower() in div_text.lower():
                            await li.click()
                            await asyncio.sleep(0.2)
                            return True
            break
    
    # If not found, close the listbox
    # await input_el.keyboard.press('Escape')
    return False
    
async def get_element_options(input_el, page, input_tag, input_type, full_page):
    """Get options for different input types"""
    options = None
    
    if input_tag == "button" or await input_el.get_attribute('role') == 'combobox':
        # Handle listbox/combobox buttons
        aria_haspopup = await input_el.get_attribute('aria-haspopup')
        if aria_haspopup == "listbox":
            options = await get_listbox_options(input_el, page, full_page)
    
    return options

async def fill_element(full_page, input_el, input_id, input_type, input_tag, response, options=None):
    try:
        if response == "SKIP":
            print(f"Skipping input {input_id} as per AI {response}")
            return

        if input_tag == "input" and input_type == "file":
            print(" --DEBUG-- Filling file upload input")
            await input_el.set_input_files([response])
            print("File input filled successfully")
            return

        # TEXT inputs or TEXTAREAs
        if input_tag in ["input", "textarea"] and not await input_el.get_attribute("dir") and input_type not in ["radio", "checkbox", "spinbutton"]:
            print(f"Filling input {input_id} with response: {response}")
            await input_el.fill(response)
            print(f"Input filled successfully")
            return

        # This is for multiInputContainer elements
        elif input_tag in ["input", "textarea"] and await input_el.get_attribute("dir") and input_type not in ["radio", "checkbox", "spinbutton"]:
            
            # if response is a list
            if isinstance(response, list):
                pass
            else:
                response = [response]      
            
            for res in response:
                print(f"Filling multiInputContainer {input_id} with response: {res}")
                await input_el.click()
                await asyncio.sleep(0.5)  # Wait for focus
                await input_el.fill(res)
                await input_el.press('Enter')  # Submit the input if needed
                await asyncio.sleep(1)  # Wait for any potential UI updates
                # if there are promptOption options, select the first one
                # break
                prompt_options = await full_page.query_selector_all(
                    'div[data-automation-id="promptOption"], div[data-automation-id="promptLeafNode"]'
                )
                # the prompt_options tagName shouldn't be <p (as in paragraph) but <span> or <div>
                print("Before : ", prompt_options)
                # prompt_options = [opt for opt in prompt_options if not await opt.evaluate('(el) => el.tagName.toLowerCase()') == 'p']
                # print("After : ", prompt_options)
                if prompt_options:
                    first_option = prompt_options[0]
                    await first_option.click()
                    print(f"Selected first promptOption: {await first_option.text_content()}")
                
            print("MultiInputContainer filled successfully")

        elif input_type == "checkbox":
            is_checked = await input_el.is_checked()
            should_be_checked = str(response).lower() in ["yes", "true", "1", "check", "selected"] or response is True
            
            if should_be_checked and not is_checked:
                await input_el.click()
            elif not should_be_checked and is_checked:
                await input_el.click()
            return

        # SELECT dropdowns
        elif input_tag == "select":
            # Validate response against options if available
            if options and response not in options:
                print(f"[WARN] Response '{response}' not in select options: {options}")
                return
            await input_el.select_option(label=response)
            print(f"Select dropdown {input_id} filled successfully with response: {response}")

        # LISTBOX/COMBOBOX buttons
        elif input_tag == "button":
            print(f"Filling listbox {input_id} with response: {response}")
            role = await input_el.get_attribute('role')
            aria_haspopup = await input_el.get_attribute('aria-haspopup')
            if aria_haspopup == "listbox":
                # Validate response against options
                if options and not any(response.lower() in opt.lower() for opt in options):
                    print(f"[WARN] Response '{response}' not found in listbox options: {options}")
                    return
                print("Response found in options, proceeding to fill listbox")
                
                success = await fill_listbox_element(input_el, page, response, full_page)
                if not success:
                    print(f"[WARN] Failed to select '{response}' from listbox")
            else:
                print(f"[WARN] Unhandled button role: {role} or aria-haspopup: {aria_haspopup}")
            print(f"Listbox {input_id} filled successfully with response: {response}")

        # DATE (spinbutton)
        elif input_type == "spinbutton":
            await input_el.fill(response)
            print(f"Spinbutton {input_id} filled successfully with response: {response}")

        # CHECKBOX or RADIO
        elif input_type in ["checkbox", "radio"]:
            if response.lower() in ["yes", "true", "1", "check", "selected"]:
                is_checked = await input_el.is_checked()
                if not is_checked:
                    await input_el.click()
            print(f"Checkbox/Radio {input_id} filled successfully with response: {response}")

        else:
            print(f"[WARN] Unhandled input type/tag: {input_type} / {input_tag}")
            print(f"Input ID: {input_id}, Type: {input_type}, Tag: {input_tag}, Response: {response}")

    except Exception as e:
        print(f"[ERROR] fill_element failed for {input_tag} / {input_type}: {e}")

asyncio.sleep(5)  # Wait for any potential UI updates

main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')

INPUT_SELECTOR = 'button, input, select, textarea, [role="button"]' # Example selector

inputs = await main_page.query_selector_all(INPUT_SELECTOR)

i = 0

prev_answered_question = None

while i < len(inputs):

    input_el = inputs[i]

    # Your existing logic to process the element
    input_id = await input_el.get_attribute('data-automation-id') or await input_el.get_attribute('aria-haspopup') or 'unknown'
    group_label, aria_labelledby = await get_group_label_and_aria(input_el, page)
    question = group_label or 'UNLABELED'

    question = await get_nearest_label_text(input_el, page) or question

    if question != 'UNLABELED' and question == prev_answered_question:
        print(f"⏩ Skipping duplicate question at index {i}: '{question}'")
        i += 1
        continue

    if input_id == "pageFooterBackButton" or input_id == "backToJobPosting":
        # We just want to skip this element and move to the next one
        i += 1
        continue 

    print(f"Input ID: {input_id}, Question: {question}, aria-labelledby: {aria_labelledby or 'None'}")

    if "dateSection" in input_id and aria_labelledby == "selfIdentifiedDisabilityData-section" :

        formatted_date = date.today().strftime("%d-%m-%Y")
        day, month, year = formatted_date.split('-')
        print(f"Day: {day}, Month: {month}, Year: {year}")
        response = None
        if "day" in input_id.lower():
            input_type = "spinbutton"
            input_tag = "input"
            input_id = "selfIdentifiedDisabilityData-day"
            response = day
        elif "month" in input_id.lower():
            input_type = "spinbutton"
            input_tag = "input"
            input_id = "selfIdentifiedDisabilityData-month"
            response = month
        elif "year" in input_id.lower():
            input_type = "spinbutton"
            input_tag = "input"
            input_id = "selfIdentifiedDisabilityData-year"
            response = year

        await fill_element(
            page,
            input_el, 
            input_id, 
            input_type,
            input_tag,
            response,
            options=None
        )
        inputs = await main_page.query_selector_all(INPUT_SELECTOR)
        i += 1
        continue


    if input_id == "pageFooterNextButton":
        print("Clicking Next button")
        await input_el.click()

    # Process non-add-button elements
    else:
        user_data = await load_user_profile()
        main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')
        section = await main_page.query_selector(f'div[aria-labelledby="{aria_labelledby}"]')

        input_type = await input_el.get_attribute('type') or 'unknown'
        role = await input_el.get_attribute('role')
        placeholder = await input_el.get_attribute('placeholder')
        required = await input_el.get_attribute('required')

        if role == "spinbutton":
            input_type = "spinbutton"

        tag_name = await input_el.evaluate("(el) => el.tagName.toLowerCase()")
        if tag_name and tag_name.lower() == 'textarea':
            input_type = 'textarea'
        input_tag = tag_name
        
        # Get options for all relevant input types
        options = await get_element_options(input_el, section, input_tag, input_type, page)
        
        # Get the nearest question/label for this element
        # element_question = await get_nearest_label_text(input_el, section)

        print(f"Processing general form element: {question}")
        
        # Create element info for AI processing
        element_info = {
            'question': question or 'UNLABELED',
            'aria_labelledby': aria_labelledby,
            'input_type': input_type,
            'input_tag': input_tag,
            'input_id': input_id,
            'options': options,
            'placeholder': placeholder,
            'required': required,
            'role': role
        }
        
        print(element_info)
        
        # # Get AI response using general user data
        full_key = f"[{element_info['question']}, {element_info['input_id']}, {element_info['input_type']}, {element_info['aria_labelledby']}, {element_info['input_tag']}]"
        ai_values, _ = await get_ai_response_for_panel(user_data, [element_info], client)
        response = ai_values.get(full_key, 'SKIP')
        
        print(f"AI response for general field '{question}': {response}")

        await fill_element(
            page,
            input_el, 
            input_id, 
            input_type, 
            input_tag, 
            response,
            options
        )

        if question != 'UNLABELED':
            prev_answered_question = question

    inputs = await main_page.query_selector_all(INPUT_SELECTOR)

    i += 1

Element ID: None
Element ID: None
Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Processing general form element: UNLABELED
{'question': 'UNLABELED', 'aria_labelledby': None, 'input_type': 'unknown', 'input_tag': 'button', 'input_id': 'unknown', 'options': None, 'placeholder': None, 'required': None, 'role': None}


/var/folders/bc/7dmh10j96rscf81jsj7gbw8h0000gn/T/ipykernel_1966/2910557440.py:330: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(5)  # Wait for any potential UI updates


Raw Response:  ChatCompletion(id='chatcmpl-C0KVRv3hYbUr0casGxz5swAhuLJXH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "[UNLABELED, unknown, unknown, None, button]": "yes"\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1754194861, model='o4-mini-2025-04-16', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=231, prompt_tokens=2245, total_tokens=2476, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=192, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=1920)))
AI Response Content:  {
  "[UNLABELED, unknown, unknown, None, button]": "yes"
}
Parsed AI Response:  {'[UNLABELED, unknown, unknown, None, button]': 'yes'}
AI response for general field 'UNLABELED': yes
Filling listbox unkno

# Next

In [ ]:
await asyncio.sleep(5)  # Wait for any potential UI updates

async def get_nearest_label_text(element, page):
    """Get the nearest label text for a form element by checking multiple DOM patterns"""
    try:
        # Try explicit <label for="">
        element_id = await element.get_attribute('id')
        if element_id and element_id != "unknown":
            label_elem = await page.query_selector(f'label[for="{element_id}"]')
            if label_elem:
                label_text = await label_elem.text_content()
                if label_text:
                    return label_text.replace('*', '').strip()
        
        # Try parent label (element wrapped in label)
        parent_label_handle = await element.evaluate_handle('el => el.closest("label")')
        parent_label = parent_label_handle.as_element() if parent_label_handle else None
        if parent_label:
            label_text = await parent_label.text_content()
            if label_text:
                return label_text.replace('*', '').strip()
        
        # Try nearest formField label in parent containers
        form_field_label = await element.evaluate('''
            el => {
                let cur = el.parentElement;
                let depth = 0;
                while (cur && depth < 10) {
                    if (cur.tagName.toLowerCase() === "div" &&
                        cur.getAttribute("data-automation-id")?.startsWith("formField-")) {
                        const lbl = cur.querySelector("label span, label");
                        if (lbl && lbl.textContent) return lbl.textContent.trim();
                    }
                    cur = cur.parentElement;
                    depth++;
                }
                return null;
            }
        ''')
        if form_field_label:
            return form_field_label.replace('*', '').strip()
        
        # Try aria-labelledby attribute
        aria_labelledby = await element.get_attribute('aria-labelledby')
        if aria_labelledby:
            label_element = await page.query_selector(f'#{aria_labelledby}')
            if label_element:
                label_text = await label_element.text_content()
                if label_text:
                    return label_text.replace('*', '').strip()
        
        # Try aria-label attribute
        aria_label = await element.get_attribute('aria-label')
        if aria_label:
            return aria_label.replace('*', '').strip()
        
        # Try placeholder as fallback
        placeholder = await element.get_attribute('placeholder')
        if placeholder:
            return placeholder.replace('*', '').strip()
        
        # Try searching for nearby text nodes or labels
        nearby_label = await element.evaluate('''
            el => {
                // Check previous sibling elements for labels
                let prev = el.previousElementSibling;
                while (prev) {
                    if (prev.tagName.toLowerCase() === 'label' || prev.textContent.trim()) {
                        return prev.textContent.trim();
                    }
                    prev = prev.previousElementSibling;
                }
                
                // Check parent's previous siblings
                let parent = el.parentElement;
                if (parent) {
                    let prevParent = parent.previousElementSibling;
                    while (prevParent) {
                        const label = prevParent.querySelector('label');
                        if (label && label.textContent) {
                            return label.textContent.trim();
                        }
                        if (prevParent.textContent.trim()) {
                            return prevParent.textContent.trim();
                        }
                        prevParent = prevParent.previousElementSibling;
                    }
                }
                
                return null;
            }
        ''')
        if nearby_label:
            return nearby_label.replace('*', '').strip()
        
        return None
        
    except Exception as e:
        print(f"Error getting label for element: {e}")
        return None

main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')

inputs = await main_page.query_selector_all('input, button, textarea, select')

for input_el in inputs:
    input_id = await input_el.get_attribute('data-automation-id') or await input_el.get_attribute('aria-haspopup') or 'unknown'
    group_label, aria_labelledby = await get_group_label_and_aria(input_el, page)
    question = group_label or 'UNLABELED'

    question = await get_nearest_label_text(input_el, page) or question

    if input_id == "pageFooterBackButton" or input_id == "backToJobPosting":
        continue

    print(f"Input ID: {input_id}, Question: {question}, aria-labelledby: {aria_labelledby or 'None'}")

Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Input ID: unknown, Question: Work Experience 1, aria-labelledby: Work-Experience-1-panel
Input ID: unknown, Question: Job Title, aria-labelledby: Work-Experience-1-panel
Input ID: unknown, Question: Company, aria-labelledby: Work-Experience-1-panel
Input ID: unknown, Question: Location, aria-labelledby: Work-Experience-1-panel
Input ID: unknown, Question: I currently work here, aria-labelledby: Work-Experience-1-panel
Input ID: dateSectionMonth-input, Question: From, aria-labelledby: Work-Experience-1-panel
Input ID: dateSectionYear-input, Question: From, aria-labelledby: Work-Experience-1-panel
Input ID: unknown, Question: Role Description, aria-labelledby: Work-Experience-1-panel
Input ID: unknown, Question: Work Experience 2, aria-labelledby: Work-Experience-2-panel
Input ID: unknown, Question: Job Title, aria-labelledby: Work-Experience-2-panel
Input ID: unkno

In [ ]:
from datetime import date

async def get_nearest_label_text(element, page):
    # Try explicit <label for="">
    element_id = await element.get_attribute('id')
    print(f"Element ID: {element_id}")
    if element_id != "unknown":
        label_elem = await page.query_selector(f'label[for="{element_id}"]')
        if label_elem:
            label_text = await label_elem.text_content()
            if label_text:
                return label_text.strip()
    # Try parent label
    parent_label_handle = await element.evaluate_handle('el => el.closest("label")')
    parent_label = parent_label_handle.as_element() if parent_label_handle else None
    if parent_label:
        label_text = await parent_label.text_content()
        if label_text:
            return label_text.strip()
    # Try nearest formField label
    form_field_label = await element.evaluate('''
        el => {
            let cur = el.parentElement;
            let depth = 0;
            while (cur && depth < 10) {
                if (cur.tagName.toLowerCase() === "div" &&
                    cur.getAttribute("data-automation-id")?.startsWith("formField-")) {
                    const lbl = cur.querySelector("label span, label");
                    if (lbl && lbl.textContent) return lbl.textContent.trim();
                }
                cur = cur.parentElement;
                depth++;
            }
            return null;
        }
    ''')
    if form_field_label:
        return form_field_label.strip()
    # Fallback: None
    return None

async def get_ai_response_for_panel(current_data, panel_elements, client):
    # Construct the list of questions for the panel with complete element info
    form_fields = []
    key_mapping = {}  # Add this to track the mapping

    for el in panel_elements:
        # Create the full key format that AI will use
        full_key = f"[{el['question']}, {el['input_id']}, {el['input_type']}, {el['aria_labelledby']}, {el['input_tag']}]"
        
        form_fields.append({
            "full_key": full_key,  # Add the full key to the form field data
            "question": el['question'],
            "input_id": el['input_id'],
            "input_type": el['input_type'],
            "input_tag": el['input_tag'],
            "aria_labelledby": el['aria_labelledby'],
            "options": el['options'], 
            "placeholder": el.get('placeholder'),
            "required": el.get('required'),
            "role": el.get('role')
        })
        
        # Store mapping from full key to element
        key_mapping[full_key] = el

    prompt = f"""
You are helping fill a job application form.
You are mapping user profile data to a web form.

You are given:
- An Entry from user profile data (JSON)
- A list of form fields from the application panel (including labels, field types, and available options if there is dropdown for the element)

Return a JSON dictionary mapping the EXACT "full_key" values to appropriate values. Use the user data to fill the values. If a field is not relevant, map it to "SKIP".

CRITICAL: You MUST use the EXACT "full_key" value as the key in your response JSON. Do NOT use just the question text.

IMPORTANT RULES:
- For fields with "options" not None:
  - You MUST select ONLY from the list of provided OPTIONS (case-sensitive)
  - If the user data is longer (e.g., "Bachelor of Engineering in Computer Science") and options are shorter (e.g., "BS"), choose the CLOSEST MATCH based on meaning
  - If no match is appropriate, use "SKIP"
- For date fields: Month should be number format (e.g., "01" for January), year should be "YYYY" format
- For date-related fields (e.g. type="spinbutton" or input_id includes "Month" or "Year"):
  - Use "MM" format for months (e.g., "01" for January)
  - Use "YYYY" format for years (e.g., "2022")
  - Match "start date", "end date", "graduation date", etc., with the corresponding data from user profile
- For text fields: Keep responses concise and relevant
- Match options exactly as they appear in the options list (case-sensitive) when options is not None
- After filling the form, if a field for save and continue is present, respond with yes to save the form

SPECIAL HANDLING FOR SKILLS/MULTI-VALUE FIELDS:
- For fields related to skills, technologies, competencies, or any field that should contain multiple items:
  - Return an ARRAY of strings instead of a single comma-separated string
  - Each skill/technology should be a separate string in the array
  - Example: ["C#", "TypeScript", "Java", "SQL", "HTML5", "CSS3", "Python"] instead of "C#, TypeScript, Java, SQL, HTML5, CSS3, Python"
- Identify skills fields by keywords in the question like: "skills", "technologies", "competencies", "tools", "programming languages", etc.

Data from User Profile:
{json.dumps(current_data, indent=2)}

Form Fields:
{json.dumps(form_fields, indent=2)}

Example response format:
{{
  "[School or University*, unknown, text, Education-(Optional)-2-panel, input]": "University Name",
  "[Degree*, unknown, button, Education-(Optional)-2-panel, button]": "MS",
  "[Field of Study, unknown, unknown, Education-(Optional)-2-panel, input]": "Computer Science",
  "[Type to Add Skills, unknown, unknown, Skills-section, input]": ["C#", "TypeScript", "Java", "SQL", "HTML5", "CSS3", "Python", ".NET Core", "Angular 2+", "RxJS", "Entity Framework", "React", "Redux", "Bootstrap 4"]
}}

Respond ONLY with a valid JSON object using the exact "full_key" values as keys.
"""
    
    try:
        response = await client.chat.completions.create(
            model="o4-mini",
            messages=[{"role": "user", "content": prompt}]
            # temperature=0.2,
            # max_tokens=500,
        )
        print("Raw Response: ", response)
        content = response.choices[0].message.content.strip()
        print("AI Response Content: ", content)
        ai_response = json.loads(content)
        print("Parsed AI Response: ", ai_response)
        return ai_response, key_mapping
    except Exception as e:
        print(f"Error in get_ai_response_for_panel: {e}")
        return {}, {}
    return {},{}

async def get_listbox_options(input_el, page, full_page):
    """Extract options from a listbox by clicking it and reading the <li> elements"""
    print("Starting to get listbox options")
    await input_el.click()
    await asyncio.sleep(1)  # Wait for options to appear
    
    # Look for the opened listbox container with visibility="opened"
    selector = 'div[visibility="opened"]'

    options = []
    listbox_container = await full_page.query_selector(selector)
    
    if listbox_container:
        li_elements = await listbox_container.query_selector_all('li[role="option"]')

        for li in li_elements:
            # Get text content, which will extract from nested elements like <div>
            text = await li.text_content()
            if text and text.strip():
                options.append(text.strip())
            else:
                # Fallback: try to get text from nested div if direct text_content fails
                div_text = await li.query_selector('div')
                if div_text:
                    nested_text = await div_text.text_content()
                    if nested_text and nested_text.strip():
                        options.append(nested_text.strip())

    # Close the listbox after getting options
    await input_el.click()
    await asyncio.sleep(0.5)
    print("Options : ", options)
    return options

async def fill_listbox_element(input_el, page, response, full_page):
    """Fill a listbox/combobox element with the given response"""
    # Click to open the listbox
    await input_el.click()
    await asyncio.sleep(0.5)
    
    # Look for the opened listbox with the specific visibility attribute
    listbox_selectors = [
        'div[visibility="opened"]',
    ]
    
    for selector in listbox_selectors:
        listbox = await full_page.query_selector(selector)
        if listbox:
            li_elements = await listbox.query_selector_all('li')
            for li in li_elements:
                # Get text content from li or nested elements
                text = await li.text_content()
                print(text.lower())
                print(response.lower())
                if text.lower() == response.lower():
                    await li.click()
                    # await asyncio.sleep(0.2)
                    return True
                else:
                    # Fallback: check nested div content
                    div_element = await li.query_selector('div')
                    if div_element:
                        div_text = await div_element.text_content()
                        if div_text and response.lower() in div_text.lower():
                            await li.click()
                            await asyncio.sleep(0.2)
                            return True
            break
    
    # If not found, close the listbox
    # await input_el.keyboard.press('Escape')
    return False
    
async def get_element_options(input_el, page, input_tag, input_type, full_page):
    """Get options for different input types"""
    options = None
    
    if input_tag == "button" or await input_el.get_attribute('role') == 'combobox':
        # Handle listbox/combobox buttons
        aria_haspopup = await input_el.get_attribute('aria-haspopup')
        if aria_haspopup == "listbox":
            options = await get_listbox_options(input_el, page, full_page)
    
    return options

async def fill_element(full_page, input_el, input_id, input_type, input_tag, response, options=None):
    try:
        if response == "SKIP":
            print(f"Skipping input {input_id} as per AI {response}")
            return

        if input_tag == "input" and input_type == "file":
            print(" --DEBUG-- Filling file upload input")
            await input_el.set_input_files([response])
            print("File input filled successfully")
            return

        # TEXT inputs or TEXTAREAs
        if input_tag in ["input", "textarea"] and not await input_el.get_attribute("dir") and input_type not in ["radio", "checkbox", "spinbutton"]:
            print(f"Filling input {input_id} with response: {response}")
            await input_el.fill(response)
            print(f"Input filled successfully")
            return

        # This is for multiInputContainer elements
        elif input_tag in ["input", "textarea"] and await input_el.get_attribute("dir") and input_type not in ["radio", "checkbox", "spinbutton"]:
            
            # if response is a list
            if isinstance(response, list):
                pass
            else:
                response = [response]      
            
            for res in response:
                print(f"Filling multiInputContainer {input_id} with response: {res}")
                await input_el.click()
                await asyncio.sleep(0.5)  # Wait for focus
                await input_el.fill(res)
                await input_el.press('Enter')  # Submit the input if needed
                await asyncio.sleep(1)  # Wait for any potential UI updates
                # if there are promptOption options, select the first one
                # break
                prompt_options = await full_page.query_selector_all(
                    'div[data-automation-id="promptOption"], div[data-automation-id="promptLeafNode"]'
                )
                # the prompt_options tagName shouldn't be <p (as in paragraph) but <span> or <div>
                print("Before : ", prompt_options)
                # prompt_options = [opt for opt in prompt_options if not await opt.evaluate('(el) => el.tagName.toLowerCase()') == 'p']
                # print("After : ", prompt_options)
                if prompt_options:
                    first_option = prompt_options[0]
                    await first_option.click()
                    print(f"Selected first promptOption: {await first_option.text_content()}")
                
            print("MultiInputContainer filled successfully")

        elif input_type == "checkbox":
            is_checked = await input_el.is_checked()
            should_be_checked = str(response).lower() in ["yes", "true", "1", "check", "selected"] or response is True
            
            if should_be_checked and not is_checked:
                await input_el.click()
            elif not should_be_checked and is_checked:
                await input_el.click()
            return

        # SELECT dropdowns
        elif input_tag == "select":
            # Validate response against options if available
            if options and response not in options:
                print(f"[WARN] Response '{response}' not in select options: {options}")
                return
            await input_el.select_option(label=response)
            print(f"Select dropdown {input_id} filled successfully with response: {response}")

        # LISTBOX/COMBOBOX buttons
        elif input_tag == "button":
            print(f"Filling listbox {input_id} with response: {response}")
            role = await input_el.get_attribute('role')
            aria_haspopup = await input_el.get_attribute('aria-haspopup')
            if aria_haspopup == "listbox":
                # Validate response against options
                if options and not any(response.lower() in opt.lower() for opt in options):
                    print(f"[WARN] Response '{response}' not found in listbox options: {options}")
                    return
                print("Response found in options, proceeding to fill listbox")
                
                success = await fill_listbox_element(input_el, page, response, full_page)
                if not success:
                    print(f"[WARN] Failed to select '{response}' from listbox")
            else:
                print(f"[WARN] Unhandled button role: {role} or aria-haspopup: {aria_haspopup}")
            print(f"Listbox {input_id} filled successfully with response: {response}")

        # DATE (spinbutton)
        elif input_type == "spinbutton":
            await input_el.fill(response)
            print(f"Spinbutton {input_id} filled successfully with response: {response}")

        # CHECKBOX or RADIO
        elif input_type in ["checkbox", "radio"]:
            if response.lower() in ["yes", "true", "1", "check", "selected"]:
                is_checked = await input_el.is_checked()
                if not is_checked:
                    await input_el.click()
            print(f"Checkbox/Radio {input_id} filled successfully with response: {response}")

        else:
            print(f"[WARN] Unhandled input type/tag: {input_type} / {input_tag}")
            print(f"Input ID: {input_id}, Type: {input_type}, Tag: {input_tag}, Response: {response}")

    except Exception as e:
        print(f"[ERROR] fill_element failed for {input_tag} / {input_type}: {e}")

asyncio.sleep(5)  # Wait for any potential UI updates

main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')

INPUT_SELECTOR = 'button, input, select, textarea, [role="button"]' # Example selector

inputs = await main_page.query_selector_all(INPUT_SELECTOR)

i = 0

prev_answered_question = None

while i < len(inputs):

    input_el = inputs[i]

    # Your existing logic to process the element
    input_id = await input_el.get_attribute('data-automation-id') or await input_el.get_attribute('aria-haspopup') or 'unknown'
    group_label, aria_labelledby = await get_group_label_and_aria(input_el, page)
    question = group_label or 'UNLABELED'

    question = await get_nearest_label_text(input_el, page) or question

    if question != 'UNLABELED' and question == prev_answered_question:
        print(f"⏩ Skipping duplicate question at index {i}: '{question}'")
        i += 1
        continue

    if input_id == "pageFooterBackButton" or input_id == "backToJobPosting":
        # We just want to skip this element and move to the next one
        i += 1
        continue 

    print(f"Input ID: {input_id}, Question: {question}, aria-labelledby: {aria_labelledby or 'None'}")

    if "dateSection" in input_id and aria_labelledby == "selfIdentifiedDisabilityData-section" :

        formatted_date = date.today().strftime("%d-%m-%Y")
        day, month, year = formatted_date.split('-')
        print(f"Day: {day}, Month: {month}, Year: {year}")
        response = None
        if "day" in input_id.lower():
            input_type = "spinbutton"
            input_tag = "input"
            input_id = "selfIdentifiedDisabilityData-day"
            response = day
        elif "month" in input_id.lower():
            input_type = "spinbutton"
            input_tag = "input"
            input_id = "selfIdentifiedDisabilityData-month"
            response = month
        elif "year" in input_id.lower():
            input_type = "spinbutton"
            input_tag = "input"
            input_id = "selfIdentifiedDisabilityData-year"
            response = year

        await fill_element(
            page,
            input_el, 
            input_id, 
            input_type,
            input_tag,
            response,
            options=None
        )
        inputs = await main_page.query_selector_all(INPUT_SELECTOR)
        i += 1
        continue


    if input_id == "pageFooterNextButton":
        await input_el.click()

    # Process non-add-button elements
    else:
        user_data = await load_user_profile()
        main_page = await page.query_selector('div[data-automation-id="applyFlowPage"]')
        section = await main_page.query_selector(f'div[aria-labelledby="{aria_labelledby}"]')

        input_type = await input_el.get_attribute('type') or 'unknown'
        role = await input_el.get_attribute('role')
        placeholder = await input_el.get_attribute('placeholder')
        required = await input_el.get_attribute('required')

        if role == "spinbutton":
            input_type = "spinbutton"

        tag_name = await input_el.evaluate("(el) => el.tagName.toLowerCase()")
        if tag_name and tag_name.lower() == 'textarea':
            input_type = 'textarea'
        input_tag = tag_name
        
        # Get options for all relevant input types
        options = await get_element_options(input_el, section, input_tag, input_type, page)
        
        # Get the nearest question/label for this element
        # element_question = await get_nearest_label_text(input_el, section)

        print(f"Processing general form element: {question}")
        
        # Create element info for AI processing
        element_info = {
            'question': question or 'UNLABELED',
            'aria_labelledby': aria_labelledby,
            'input_type': input_type,
            'input_tag': input_tag,
            'input_id': input_id,
            'options': options,
            'placeholder': placeholder,
            'required': required,
            'role': role
        }
        
        print(element_info)
        
        # # Get AI response using general user data
        full_key = f"[{element_info['question']}, {element_info['input_id']}, {element_info['input_type']}, {element_info['aria_labelledby']}, {element_info['input_tag']}]"
        ai_values, _ = await get_ai_response_for_panel(user_data, [element_info], client)
        response = ai_values.get(full_key, 'SKIP')
        
        print(f"AI response for general field '{question}': {response}")

        await fill_element(
            page,
            input_el, 
            input_id, 
            input_type, 
            input_tag, 
            response,
            options
        )

        if question != 'UNLABELED':
            prev_answered_question = question

    inputs = await main_page.query_selector_all(INPUT_SELECTOR)

    i += 1

/var/folders/bc/7dmh10j96rscf81jsj7gbw8h0000gn/T/ipykernel_1966/3736541739.py:329: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(5)  # Wait for any potential UI updates


Element ID: None
Element ID: None
Input ID: unknown, Question: UNLABELED, aria-labelledby: None
Processing general form element: UNLABELED
{'question': 'UNLABELED', 'aria_labelledby': None, 'input_type': 'unknown', 'input_tag': 'button', 'input_id': 'unknown', 'options': None, 'placeholder': None, 'required': None, 'role': None}
Raw Response:  ChatCompletion(id='chatcmpl-C0KaH7kuEz4amvEkpPtRJ0FRgjY1N', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "[UNLABELED, unknown, unknown, None, button]": "yes"\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1754195161, model='o4-mini-2025-04-16', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=167, prompt_tokens=2221, total_tokens=2388, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=128, rejec

CancelledError: 